# Latent Customer Behaviour Segmentation Implementation Pipeline

**Pipeline Configuration:**
**data -> embedding features -> clustering -> evaluation -> explainability -> business‑friendly profiles**.


## 0. Dependency Installation

In [1]:
# Installing necessary libraries

import os
import sys
from pathlib import Path

# Core scientific + ML stack libraries
%pip install -q scipy pandas scikit-learn gensim umap-learn hdbscan shap xgboost seaborn tqdm pyarrow polars matplotlib numpy

# Extra utilities being used
%pip install -q tqdm-joblib
%pip install -q --upgrade kaggle
%pip install -q ipython-autotime

# Load autotime extension to measure execution times automatically
%load_ext autotime

print("Dependencies installed successfully")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Dependencies installed successfully
time: 0 ns (started: 2025-08-27 14:40:50 +01:00)


## 1. Runtime Configuration and Data: Loading, Basic Checks, and Sessionisation

In [2]:
# 0) CONFIGURATIONS

from pathlib import Path
import os
import numpy as np
import random
import datetime as dt
import pandas as pd

# Project meta
PROJECT_NAME = "Latent_Customer_Behaviour_Segmentation_Implementation_Pipeline"
RUN_ID       = dt.datetime.now().strftime("%Y%m%d_%H%M%S")  # used in file names

# Reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Threads / parallelisation
N_JOBS   = -1   # -1 = use all cores-1; set to fixed int for reproducibility

#  Paths
ROOT_DIR   = Path("C:/Users/Admin/Documents/WBS/Dissertation/Submission Related files/Notebooks/Modelling")
DATA_DIR   = ROOT_DIR / "data"
OUT_DIR    = ROOT_DIR / "outputs"
FIGS_DIR   = OUT_DIR / "figs"
TABLES_DIR = OUT_DIR / "tables"
SHAP_DIR   = OUT_DIR / "shap_artifacts"

# Input files
EVENTS_CSV = DATA_DIR / "events.csv"               # clickstream (session_id, user_id, item_id, timestamp, event, etc.)
ITEMS_CSV  = DATA_DIR / "item_properties_part1.csv"   # optional (item metadata)
ITEMS2_CSV = DATA_DIR / "item_properties_part2.csv"        # optional 2nd metadata file

# Ensure output folders exist
for p in [OUT_DIR, FIGS_DIR, TABLES_DIR, SHAP_DIR]:
    p.mkdir(parents=True, exist_ok=True)

# Data schema & timestamp handling
# Only change these if your raw file uses different column names
COLS = {
    "session":   "session_id",
    "user":      "user_id",
    "item":      "item_id",
    "timestamp": "timestamp",
    "event":     "event"          # expected: "view", "addtocart", "transaction" etc.
}
# timestamps sometimes come in ms; if so, set TS_DIV=1000 to convert to seconds
TS_DIV = 1000  # 1 if already epoch seconds; 1000 if epoch milliseconds

# Session construction
SESSION_GAP_SEC       = 30 * 60   # break session if gap > 30 minutes
MIN_EVENTS_PER_SESSION= 2         # drop sessions with fewer events than this
MAX_EVENTS_PER_SESSION= 5000      # guardrail for bots/anomalies
DROP_ABNORMAL_USERS   = True      # remove outliers by volume (top x%)
ABNORMAL_USER_PCTL    = 99.9      # drop users above this percentile of event count

#  Representation / Embeddings
# We combine session-level Doc2Vec + item-level Word2Vec (mean-pooled)
USE_DOC2VEC           = True
USE_WORD2VEC          = True
CONCAT_DOC_W2V        = True

# Extra options
USE_SIF_WEIGHTING   = True   # Smooth Inverse Frequency weighting for W2V pooling
USE_TFIDF_WEIGHTING = False  # TF-IDF pooling instead of SIF (rarely needed)
USE_DBOW_EXTRA      = True   # Add small DBOW Doc2Vec for robustness

# Word2Vec (items -> vector)
W2V_VECTOR_SIZE       = 64
W2V_WINDOW            = 5
W2V_MIN_COUNT         = 2
W2V_SG                = 1          # 1 = skip-gram, 0 = CBOW
W2V_NEGATIVE          = 5
W2V_SUBSAMPLE         = 1e-5
W2V_EPOCHS            = 20

# Doc2Vec (session sequences -> vector)
D2V_VECTOR_SIZE       = 96
D2V_WINDOW            = 5
D2V_MIN_COUNT         = 2
D2V_DM                = 1          # 1 = PV-DM, 0 = PV-DBOW
D2V_EPOCHS            = 30
D2V_ALPHA             = 0.025
D2V_MIN_ALPHA         = 0.0005

# Dimensionality reduction for modelling & viz
USE_PCA_FOR_MODEL     = True
PCA_N_COMPONENTS      = 96          # for downstream clustering (denoising)
PCA_WHITEN            = False       # Whitening distorts the natural variance structure, reducing the ability to interpret PCA components by how much variance they explain
PCA_RANDOM_STATE      = RANDOM_SEED

USE_UMAP_FOR_VIZ      = True        # 2D viz only (do NOT use UMAP space for metrics)
UMAP_N_NEIGHBORS      = 30
UMAP_MIN_DIST         = 0.25
UMAP_N_COMPONENTS     = 2
UMAP_METRIC           = "cosine"
UMAP_RANDOM_STATE     = RANDOM_SEED

# Clustering (we compare multiple models)

# Clustering orchestration (sampling + eval)
USE_SAMPLED_CLUSTERING = True     # if False, fit all models on full data
EVAL_ON_SAMPLE         = True     # compute internal metrics on a shared sample
DO_INTERNAL_SCORES     = True     # print/save silhouette/CH/DB for quick comparisons

# MiniBatch K-Means (baseline centroid method)
KMEANS_K_GRID         = [2, 3, 4, 6, 8, 10, 12, 14]
KMEANS_BATCH_SIZE     = 4096
KMEANS_MAX_ITER       = 300
KMEANS_N_INIT          = 50       # number of centroid initialisations

# Gaussian Mixture Models (select K by BIC)
GMM_K_GRID            = [8, 10, 12, 14, 16, 188]
GMM_COV_TYPE          = "diag"
GMM_MAX_ITER          = 300
GMM_N_INIT            = 3
GMM_RANDOM_STATE      = RANDOM_SEED

# HDBSCAN (density-based; primary for segmentation)
# We run on a stratified sample for speed, then propagate labels to full set
HDBSCAN_SAMPLE_SIZE         = 90_000      # sample size for fit
HDBSCAN_MIN_CLUSTER_SIZE    = 1500         # smallest segment we care about (tune 300–1000)
HDBSCAN_MIN_SAMPLES         = 20          # higher -> stricter core definition (more noise)
HDBSCAN_METRIC              = "euclidean"    # try: "cosine" or "manhattan" (often better than euclidean for embeddings)
HDBSCAN_CLUSTER_SELECTION   = "eom"       # "eom" (stable, fewer clusters) or "leaf" (more fine-grained)
HDBSCAN_SELECTION_EPSILON   = 0.0         # >0 merges very close clusters; keep 0 unless over-fragmenting
HDBSCAN_ALLOW_SINGLE_CLUSTER= False
HDBSCAN_CLUSTER_PROB_THRESH = 0.0         # keep all; raise to 0.15–0.3 to drop low-confidence members
HDBSCAN_APPROX_PREDICT      = True        # assign remaining points to nearest clusters

# Agglomerative + kNN propagation
USE_AGGLO              = True
AGGLOMERATIVE_NCL      = 12       # number of clusters for Ward linkage (tune as needed)
K_CONNECTIVITY         = 30       # k for building connectivity graph (Ward requires euclidean)
K_PROPAGATE            = 30       # k for kNN label propagation to full set

# DBSCAN (classic)
USE_DBSCAN                  = True
DBSCAN_EPS                  = None        # None to evaluate from k-distance elbow (or evaluated from previous calibrations)
DBSCAN_MIN_SAMPLES          = 8
DBSCAN_METRIC               = "euclidean"
# DBSCAN sampling for eps estimation and fit
DBSCAN_EPS_SAMPLE_N    = 25_000  # sample size for k-distance eps estimation
DBSCAN_FIT_SAMPLE_N    = 30_000  # sample size for DBSCAN fit when sampling + propagation

# OPTICS (alternative density method)
USE_OPTICS                  = False
OPTICS_MIN_SAMPLES          = 20
OPTICS_XI                   = 0.05         # smaller -> more clusters; larger -> fewer clusters
OPTICS_MIN_CLUSTER_SIZE     = 50
OPTICS_MAX_EPS              = np.inf       # set to a reasonable cap to speed up (e.g., 2.0) if known
OPTICS_METRIC               = "cosine"

# Evaluation (internal metrics + stability)
EVAL_SAMPLE_SIZE            = 70_000       # for fast metric computation on large sets
COMPUTE_SILHOUETTE          = True         # only on non-noise labels
COMPUTE_CH_DB               = True         # Calinski–Harabasz & Davies–Bouldin
COMPUTE_DBCV                = False         # density-based validity (better for HDBSCAN)
NOISE_LABEL                 = -1

# Temporal stability (early vs late split)
CHECK_TEMPORAL_DRIFT        = True
EARLY_LATE_SPLIT_PCT        = 0.5          # 0.5 = first half vs second half by time

# Cross-model agreement (optional)
COMPUTE_ARI_AMI             = True

# Output paths for clustering evaluations
def run_stem(name: str) -> Path:
    return OUT_DIR / f"{RUN_ID}_{name}"
SCORECARD_OUT          = run_stem("cluster_scorecard.csv")

# Profiling & naming clusters
# These are simple session-level features for interpretation/profiling tables
PROFILE_FEATURES = [
    "events", "view", "addtocart", "transaction",
    "duration_sec", "events_per_min", "item_diversity",
    "last_evt_view", "last_evt_addtocart", "last_evt_transaction",
    "hour", "dow", "is_evening"
]

# RFM-style baseline (for a simple, transparent benchmark)
BASELINE_LEVEL              = "session"    # or "visitor"
MONETARY_MODE               = "transactions"  # "transactions" or "none"
K_GRID_BASE                 = [4, 6, 8, 10, 12]
SIL_SAMPLE_N = 5_000
ATC_WEIGHT   = 0.5   # add-to-cart counts as “half” a purchase in M proxy

#  SHAP (surrogate explainability for cluster membership)
USE_SHAP_SURROGATE          = True
SHAP_SAMPLE_SIZE            = 30_000      # sample for surrogate modelling
SHAP_TOP_N_FEATURES         = 15           # report top-N features globally + per-cluster
RF_N_ESTIMATORS             = 500
RF_MAX_DEPTH                = 12
RF_RANDOM_STATE             = RANDOM_SEED
SHAP_BACKGROUND_SIZE        = 2_000        # interventional background size for TreeSHAP
SAVE_BEESWARM_PNG           = True

# Saving and file naming
SAVE_CSV_SUMMARIES          = True
SAVE_VIZ_PNG                = True
FIG_DPI                     = 150

#  Utility: consistent file stem for this run
def run_stem(name: str) -> Path:
    """Create a consistent file stem under outputs/ with run id and name."""
    return OUT_DIR / f"{RUN_ID}_{name}"

# printing configuration summary for current run
def print_config_summary():
    print("CONFIGURATION SUMMARY")
    print(f"PROJECT_NAME: {PROJECT_NAME}")
    print(f"RUN_ID:       {RUN_ID}")
    print(f"Random seed:  {RANDOM_SEED}")
    print(f"Events CSV:   {EVENTS_CSV}")
    print(f"PCA dims:     {PCA_N_COMPONENTS} | PCA for model: {USE_PCA_FOR_MODEL}")
    print(f"UMAP for viz: {USE_UMAP_FOR_VIZ} (n={UMAP_N_COMPONENTS}, metric={UMAP_METRIC})")
    print(f"HDBSCAN: min_cluster_size={HDBSCAN_MIN_CLUSTER_SIZE}, min_samples={HDBSCAN_MIN_SAMPLES}, "
          f"metric={HDBSCAN_METRIC}, selection={HDBSCAN_CLUSTER_SELECTION}, "
          f"epsilon={HDBSCAN_SELECTION_EPSILON}, sample={HDBSCAN_SAMPLE_SIZE}")
    print(f"OPTICS: use={USE_OPTICS}, xi={OPTICS_XI}, min_samples={OPTICS_MIN_SAMPLES}, max_eps={OPTICS_MAX_EPS}")
    print(f"DBSCAN: use={USE_DBSCAN}, eps={DBSCAN_EPS}, min_samples={DBSCAN_MIN_SAMPLES}, metric={DBSCAN_METRIC}")
    print(f"Eval sample:  {EVAL_SAMPLE_SIZE}, DBCV={COMPUTE_DBCV}, silhouette={COMPUTE_SILHOUETTE}")
    print(f"SHAP: use={USE_SHAP_SURROGATE}, sample={SHAP_SAMPLE_SIZE}, topN={SHAP_TOP_N_FEATURES}")
    print(f"Outputs -> {OUT_DIR}")


print_config_summary()


# Loading data 

print("Loading events and building sessions")

# 1) Load events (minimal schema)

usecols = ["visitorid", "timestamp", "event", "itemid"]
events = pd.read_csv(EVENTS_CSV, usecols=usecols, nrows=None)

# Basic cleaning:
# - drop rows missing core fields
# - keep timestamps as int64 (UNIX seconds)
events = events.dropna(subset=["visitorid", "timestamp", "event"]).copy()

# If timestamps are in milliseconds, downscale to seconds (TS_DIV from CONFIG)
ts = events["timestamp"].astype("int64")
if ts.max() > 10**11 or TS_DIV == 1000:  # rough check + config guardrail
    events["timestamp"] = (ts // 1000).astype("int64")
else:
    events["timestamp"] = ts

# Normalise event labels (keeps downstream logic simple)
events["event"] = (
    events["event"]
    .astype("string")
    .str.strip()
    .str.lower()
    .replace({
        "add-to-cart": "addtocart",
        "add_to_cart": "addtocart",
        "cart": "addtocart",
        "purchase": "transaction",
        "buy": "transaction",
    })
)

# Sorting is important for session building
events = events.sort_values(["visitorid", "timestamp"]).reset_index(drop=True)

# 2) Build sessions (30 min inactivity gap)

gap = SESSION_GAP_SEC 
# New session when gap between consecutive events for the same visitor > threshold
dt = events.groupby("visitorid")["timestamp"].diff().fillna(gap + 1)
is_new_session = (dt > gap) | events["visitorid"].ne(events["visitorid"].shift())
events["session_id"] = np.cumsum(is_new_session)

# Dropping super-short sessions (e.g., single-click noise)
if MIN_EVENTS_PER_SESSION and MIN_EVENTS_PER_SESSION > 1:
    sizes = events.groupby("session_id").size()
    keep = sizes[sizes >= MIN_EVENTS_PER_SESSION].index
    events = events[events["session_id"].isin(keep)].reset_index(drop=True)

print(f"Rows after load/sessionise: {len(events):,}")
print(f"Sessions (pre-filter):      {events['session_id'].nunique():,}")
print(f"Visitors (pre-filter):      {events['visitorid'].nunique():,}")


# 3) Helper function to compute per-session features for profiling

def compute_session_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns one row per session_id with:
    - events, duration_sec, events_per_min
    - item_diversity (unique items)
    - last event flags (view/addtocart/transaction)
    - basic time features (hour, dow, is_evening)
    """
    agg = {
        "timestamp": ["min", "max", "count"],
        "itemid": pd.Series.nunique,
    }
    g = df.groupby("session_id").agg(agg)
    g.columns = ["ts_min", "ts_max", "events", "item_diversity"]
    g["duration_sec"] = (g["ts_max"] - g["ts_min"]).clip(lower=0)
    g["events_per_min"] = g["events"] / np.maximum(1, g["duration_sec"] / 60.0)

    # Last event per session
    last_evt = (
        df.sort_values(["session_id", "timestamp"])
          .groupby("session_id")["event"].last()
    )
    for ev in ["view", "addtocart", "transaction"]:
        g[f"last_evt_{ev}"] = (last_evt == ev).astype(int)

    # Lightweight time-of-day features based on last timestamp
    last_ts = (
        df.sort_values(["session_id", "timestamp"])
          .groupby("session_id")["timestamp"].last()
    )
    last_dt = pd.to_datetime(last_ts, unit="s", utc=True).dt.tz_convert(None)
    g["hour"] = last_dt.dt.hour
    g["dow"] = last_dt.dt.dayofweek
    g["is_evening"] = g["hour"].between(17, 22).astype(int)

    return g.reset_index()

session_feats = compute_session_features(events)


# 4) dropping abnormal users by volume
DROP_ABNORMAL_USERS = False
if DROP_ABNORMAL_USERS and "visitorid" in events.columns:
    before_users = events["visitorid"].nunique()
    user_events = events.groupby("visitorid").size()
    cutoff = np.percentile(user_events, ABNORMAL_USER_PCTL)
    heavy_users = user_events[user_events > cutoff].index

    if len(heavy_users) > 0:
        events = events.loc[~events["visitorid"].isin(heavy_users)].reset_index(drop=True)
        print(
            f"[users] Dropped {len(heavy_users):,} heavy users "
            f"(>{ABNORMAL_USER_PCTL}th pct). Users: {before_users:,} -> {events['visitorid'].nunique():,}"
        )
        # Recomputing session features (as some sessions might have vanished)
        session_feats = compute_session_features(events)


# 5) Defining outlier thresholds from data (p99 and safe floors threshold values)

def p99(series: pd.Series) -> float:
    return float(np.nanpercentile(series.values, 99))

p99_events       = p99(session_feats["events"])
p99_epm          = p99(session_feats["events_per_min"])
p99_duration_sec = p99(session_feats["duration_sec"])
p99_itemdiv      = p99(session_feats["item_diversity"])

# Defining floors helps in keeping thresholds sensible
THRESH_EVENTS_MAX   = max(100, int(p99_events))
THRESH_EPM_MAX      = max(30.0, p99_epm)
THRESH_DURATION_MAX = max(3 * 3600, int(p99_duration_sec))  # at least 3 hours
THRESH_ITEMDIV_MAX  = max(200, int(p99_itemdiv))

print("Outlier thresholds (p99 with floors):")
print(f"  events <= {THRESH_EVENTS_MAX:,}")
print(f"  events_per_min <= {THRESH_EPM_MAX:,.2f}")
print(f"  duration_sec <= {THRESH_DURATION_MAX:,}  (~{THRESH_DURATION_MAX/3600:.1f}h)")
print(f"  item_diversity <= {THRESH_ITEMDIV_MAX:,}")


# 6) Flagging and removing potential outliers

outlier_mask = (
    (session_feats["events"] > THRESH_EVENTS_MAX) |
    (session_feats["events_per_min"] > THRESH_EPM_MAX) |
    (session_feats["duration_sec"] > THRESH_DURATION_MAX) |
    (session_feats["item_diversity"] > THRESH_ITEMDIV_MAX)
)
potential_outliers = session_feats.loc[outlier_mask].copy()

before_rows = len(events)
before_sessions = events["session_id"].nunique()

kept_session_ids = set(session_feats.loc[~outlier_mask, "session_id"].values)
events = events[events["session_id"].isin(kept_session_ids)].reset_index(drop=True)

after_rows = len(events)
after_sessions = events["session_id"].nunique()

print(
    f"[sessions] Outlier removal: {before_sessions:,} -> {after_sessions:,} sessions "
    f"({before_sessions - after_sessions:,} dropped); rows {before_rows:,} -> {after_rows:,}"
)

# Keeping features aligned with filtered events
session_feats = compute_session_features(events)


# 7) Saving for audit and dashboard
# Percentiles
pct = (session_feats[["events", "events_per_min", "duration_sec", "item_diversity"]].describe(percentiles=[0.5, 0.9, 0.95, 0.99]).round(3))
pct_path = TABLES_DIR / f"{RUN_ID}_session_feature_percentiles.csv"
pct.to_csv(pct_path)

# Outliers we removed (transparency)
out_path = TABLES_DIR / f"{RUN_ID}_potential_automation_or_outliers.csv"
potential_outliers.to_csv(out_path, index=False)

# Dataset overview (post-cleaning)
overview = pd.DataFrame([{
    "rows": after_rows,
    "sessions": after_sessions,
    "visitors": events["visitorid"].nunique() if "visitorid" in events.columns else np.nan,
    "items": events["itemid"].nunique(),
    "date_start": pd.to_datetime(events["timestamp"], unit="s").min(),
    "date_end": pd.to_datetime(events["timestamp"], unit="s").max()
}])
ov_path = TABLES_DIR / f"{RUN_ID}_dataset_overview.csv"
overview.to_csv(ov_path, index=False)

# Building a simple session-level funnel (post-cleaning)
evt_flags = (
    events.assign(flag=1)
        .pivot_table(index="session_id", columns="event",
        values="flag", aggfunc="sum", fill_value=0)
)
# Ensuring all columns exist
for c in ["view", "addtocart", "transaction"]:
    if c not in evt_flags.columns:
        evt_flags[c] = 0
# Saving funnel data
funnel = pd.DataFrame({
    "stage": ["view", "addtocart", "transaction"],
    "sessions_reached": [
        int((evt_flags["view"] > 0).sum()),
        int(((evt_flags["view"] > 0) & (evt_flags["addtocart"] > 0)).sum()),
        int(((evt_flags["addtocart"] > 0) & (evt_flags["transaction"] > 0)).sum())
    ]
})
funnel["prop_of_total"] = funnel["sessions_reached"] / after_sessions
fun_path = TABLES_DIR / f"{RUN_ID}_funnel_session_level.csv"
funnel.to_csv(fun_path, index=False)

print("Session features ready and aligned with filtered data.")



CONFIGURATION SUMMARY
PROJECT_NAME: Latent_Customer_Behaviour_Segmentation_Implementation_Pipeline
RUN_ID:       20250827_144050
Random seed:  42
Events CSV:   C:\Users\Admin\Documents\WBS\Dissertation\Submission Related files\Notebooks\Modelling\data\events.csv
PCA dims:     96 | PCA for model: True
UMAP for viz: True (n=2, metric=cosine)
HDBSCAN: min_cluster_size=900, min_samples=20, metric=euclidean, selection=eom, epsilon=0.0, sample=90000
OPTICS: use=False, xi=0.05, min_samples=20, max_eps=inf
DBSCAN: use=True, eps=None, min_samples=8, metric=euclidean
Eval sample:  70000, DBCV=True, silhouette=True
SHAP: use=True, sample=30000, topN=15
Outputs -> C:\Users\Admin\Documents\WBS\Dissertation\Submission Related files\Notebooks\Modelling\outputs
Loading events and building sessions
Rows after load/sessionise: 1,377,221
Sessions (pre-filter):      382,780
Visitors (pre-filter):      315,642
[users] Dropped 314 heavy users (>99.9th pct). Users: 315,642 -> 315,328
Outlier thresholds (p99 

## 2. Deriving features via neural embeddings (session‑level)

In [3]:
# 2) Session-level neural embeddings


# 1) Imports
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.preprocessing import StandardScaler
import numpy as np
from collections import Counter
import math, os, time

# helpers for timing
def tic(section):
    print(f"\n>>> Starting: {section}")
    return time.perf_counter()

def toc(t0, section):
    dt = time.perf_counter() - t0
    print(f"completed: {section} in {dt:.2f}s")

# worker threads logic
if isinstance(N_JOBS, int) and N_JOBS == -1:
    THREADS = max(1, (os.cpu_count() or 2) - 1)
elif isinstance(N_JOBS, int) and N_JOBS > 0:
    THREADS = N_JOBS
else:
    THREADS = 1

# 2) Token sequences per session (items as strings)
t0 = tic("Preparing token sequences per session")
events_seq = events.sort_values(["session_id", "timestamp"], kind="mergesort")
session_to_items = (
    events_seq.groupby("session_id", sort=False)["itemid"]
              .apply(lambda s: [str(x) for x in s.values])
)
sentences = session_to_items.tolist()
print("token sequences per session prepared.")
toc(t0, "Preparing token sequences per session")

# 3) Frequency dictionaries (for SIF/TF-IDF pooling)
t0 = tic("Building frequency dictionaries")
global_counts = Counter(t for seq in sentences for t in seq)
N_tokens      = sum(global_counts.values())
N_docs        = len(sentences)
df_counts     = Counter(t for seq in map(set, sentences) for t in seq) if USE_TFIDF_WEIGHTING else None
toc(t0, "Building frequency dictionaries")

# SIF weight function
a_sif = 5e-4
def sif_weight(token: str) -> float:
    pw = global_counts[token] / max(1, N_tokens)
    return a_sif / (a_sif + pw)

# TF-IDF helper
def tfidf_pool(tokens, get_vec):
    if not tokens:
        return None, 0.0
    cnt = Counter(tokens)
    L = float(len(tokens))
    acc = None; wsum = 0.0
    for t, tf in cnt.items():
        vec = get_vec(t)
        if vec is None:
            continue
        idf = math.log((N_docs + 1) / (df_counts[t] + 1)) + 1.0
        w = (tf / L) * idf
        acc = (vec * w) if acc is None else (acc + vec * w)
        wsum += w
    return acc, wsum

# 4) Word2Vec + pooling
t0 = tic("Training Word2Vec + pooling")
w2v_features = None
if USE_WORD2VEC:
    w2v = Word2Vec(
        sentences=sentences,
        vector_size=W2V_VECTOR_SIZE,
        window=W2V_WINDOW,
        min_count=W2V_MIN_COUNT,
        workers=THREADS,
        sg=W2V_SG,
        negative=W2V_NEGATIVE,
        sample=W2V_SUBSAMPLE,
        epochs=W2V_EPOCHS,
        seed=RANDOM_SEED,
        hs=0
    )

    # pooling rules
    def mean_pool(tokens):
        vs = [w2v.wv[t] for t in tokens if t in w2v.wv]
        return (np.mean(vs, axis=0) if vs else None), 1.0

    def sif_pool(tokens):
        acc = None; wsum = 0.0
        for t in tokens:
            if t in w2v.wv:
                v = w2v.wv[t]
                w = sif_weight(t)
                acc = (v * w) if acc is None else (acc + v * w)
                wsum += w
        return acc, wsum

    def get_vec(token):
        return w2v.wv[token] if token in w2v.wv else None

    def pooled_vector(tokens):
        if USE_TFIDF_WEIGHTING:
            acc, wsum = tfidf_pool(tokens, get_vec)
        elif USE_SIF_WEIGHTING:
            acc, wsum = sif_pool(tokens)
        else:
            acc, wsum = mean_pool(tokens)
        if (acc is None) or (wsum <= 0):
            return np.zeros(W2V_VECTOR_SIZE, dtype="float32")
        return (acc / wsum).astype("float32")

    w2v_features = np.vstack([pooled_vector(tokens) for tokens in sentences])

print("Word2Vec training completed.")
toc(t0, "Training Word2Vec + pooling")

# 5) Doc2Vec (DM + optional DBOW)
t0 = tic("Training Doc2Vec (DM) [and optional DBOW]")
d2v_features_list = []
if USE_DOC2VEC:
    docs = [TaggedDocument(words=toks, tags=[str(sid)])
            for sid, toks in zip(session_to_items.index.values, sentences)]

    # Distributed Memory
    d2v_dm = Doc2Vec(
        documents=docs,
        vector_size=D2V_VECTOR_SIZE,
        window=D2V_WINDOW,
        min_count=D2V_MIN_COUNT,
        workers=THREADS,
        dm=D2V_DM, dm_mean=1,
        negative=W2V_NEGATIVE,
        epochs=D2V_EPOCHS,
        seed=RANDOM_SEED
    )
    d2v_dm_feats = np.vstack([d2v_dm.dv[str(sid)] for sid in session_to_items.index.values]).astype("float32")
    d2v_features_list.append(d2v_dm_feats)

    # DBOW for robustness on short sessions
    if USE_DBOW_EXTRA:
        d2v_dbow = Doc2Vec(
            documents=docs,
            vector_size=max(32, D2V_VECTOR_SIZE // 2),
            window=D2V_WINDOW,
            min_count=D2V_MIN_COUNT,
            workers=THREADS,
            dm=0, dbow_words=0,
            negative=W2V_NEGATIVE,
            epochs=max(10, D2V_EPOCHS // 2),
            seed=RANDOM_SEED
        )
        d2v_dbow_feats = np.vstack([d2v_dbow.dv[str(sid)] for sid in session_to_items.index.values]).astype("float32")
        d2v_features_list.append(d2v_dbow_feats)

d2v_features = np.hstack(d2v_features_list) if d2v_features_list else None
print("Doc2Vec training completed.")
toc(t0, "Training Doc2Vec (DM) [and optional DBOW]")

# 6) Concatenate embeddings
t0 = tic("Concatenating feature blocks")
feat_blocks = [f for f in (w2v_features, d2v_features) if f is not None]
assert len(feat_blocks) >= 1, "No embeddings produced, check USE_WORD2VEC / USE_DOC2VEC."
X = feat_blocks[0] if len(feat_blocks) == 1 else np.hstack(feat_blocks)
print("Concatenation completed. Raw feature shape:", X.shape)
toc(t0, "Concatenating feature blocks")

# 7) Standardise + L2-normalise
t0 = tic("Standardising & L2-normalising")
scaler = StandardScaler(with_mean=True, with_std=True)
X_scaled = scaler.fit_transform(X).astype("float32", copy=False)
row_norms = np.linalg.norm(X, axis=1, keepdims=True)
X_l2 = X / np.clip(row_norms, 1e-12, None)
print("Standardisation and L2-normalisation completed.")
print("Feature matrix (scaled) shape:", X_scaled.shape)
print("Feature matrix (L2) shape:", X_l2.shape)
toc(t0, "Standardising & L2-normalising")

# 8) Health checks
t0 = tic("Health checks")
n_zero_w2v = 0 if w2v_features is None else int((w2v_features == 0).all(axis=1).sum())
vocab_size = 0 if not USE_WORD2VEC else len(w2v.wv)
print("W2V vocab size:", vocab_size)
print("Zero-vector sessions from W2V pooling:", n_zero_w2v)
toc(t0, "Health checks")



>>> Starting: Preparing token sequences per session
token sequences per session prepared.
completed: Preparing token sequences per session in 8.68s

>>> Starting: Building frequency dictionaries
completed: Building frequency dictionaries in 0.30s

>>> Starting: Training Word2Vec + pooling
Word2Vec training completed.
completed: Training Word2Vec + pooling in 28.45s

>>> Starting: Training Doc2Vec (DM) [and optional DBOW]
Doc2Vec training completed.
completed: Training Doc2Vec (DM) [and optional DBOW] in 1458.71s

>>> Starting: Concatenating feature blocks
Concatenation completed. Raw feature shape: (371095, 208)
completed: Concatenating feature blocks in 0.08s

>>> Starting: Standardising & L2-normalising
Standardisation and L2-normalisation completed.
Feature matrix (scaled) shape: (371095, 208)
Feature matrix (L2) shape: (371095, 208)
completed: Standardising & L2-normalising in 1.09s

>>> Starting: Health checks
W2V vocab size: 92956
Zero-vector sessions from W2V pooling: 3646
comp

## 3. Dimensionality reduction of derived neural embeddings (PCA AND UMAP)

In [4]:
# 3) Dimensionality reduction (PCA + UMAP)

# 1) Imports
from sklearn.decomposition import PCA
import numpy as np
import umap
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# 2) PCA on scaled features (denoise + numerical stability)
X_model = X_scaled
pca_dim_eff = int(min(PCA_N_COMPONENTS, X_scaled.shape[1]))
assert pca_dim_eff >= 2, "PCA_N_COMPONENTS must be >= 2."

pca = PCA(
    n_components=pca_dim_eff,
    whiten=PCA_WHITEN,
    random_state=PCA_RANDOM_STATE
)
X_pca = pca.fit_transform(X_scaled)

# Cosine-friendly copy (L2 after PCA): Euclidean is approximately same as cosine
row_norms_pca = np.linalg.norm(X_pca, axis=1, keepdims=True)
X_pca_l2 = X_pca / np.clip(row_norms_pca, 1e-12, None)

# Downstream model inputs (choose L2 if metric is cosine)
X_model_kmeans  = X_pca_l2                            # k-means prefers L2
X_model_gmm     = X_pca                               # GMM prefers PCA/standardised space
X_model_hdbscan = X_pca_l2 if HDBSCAN_METRIC == "cosine" else X_pca
X_model_dbscan  = X_pca_l2 if DBSCAN_METRIC  == "cosine" else X_pca

# Sample sizes for any later calibration/fit subsampling
DBSCAN_SAMPLE_N  = min(90_000, X_model_dbscan.shape[0])
HDBSCAN_SAMPLE_N = min(HDBSCAN_SAMPLE_SIZE, X_model_hdbscan.shape[0])  # tied to configuration

# Reporting
expl_var = float(np.sum(pca.explained_variance_ratio_))
print(f"PCA | components: {pca_dim_eff} | total explained variance: {expl_var:.3f}")
print("k-means/HDBSCAN input shape:", X_model_kmeans.shape)
print("GMM input shape:", X_model_gmm.shape)
print("DBSCAN sample size:", DBSCAN_SAMPLE_N)
print("HDBSCAN sample size:", HDBSCAN_SAMPLE_N)

# 3) UMAP for 2D visualisation (for plots only; not used for modelling)
umap_2d = None
if USE_UMAP_FOR_VIZ:
    umap_2d = umap.UMAP(
        n_neighbors=UMAP_N_NEIGHBORS,
        min_dist=UMAP_MIN_DIST,
        n_components=2,
        metric=UMAP_METRIC,            # from config (e.g., "cosine")
        random_state=UMAP_RANDOM_STATE
    ).fit_transform(X_pca)             # run on denoised PCA space

    print("UMAP ->", umap_2d.shape)
    print("Ethical Note: UMAP is for visual inspection only; global distances/densities are not faithful.")

# 4) Save UMAP scatter to FIGS_DIR (with optional downsampling)
if USE_UMAP_FOR_VIZ and umap_2d is not None:
    rng = np.random.default_rng(RANDOM_SEED)
    UMAP_PLOT_MAX = 250_000
    n_points = umap_2d.shape[0]
    if n_points > UMAP_PLOT_MAX:
        idx = rng.choice(n_points, size=UMAP_PLOT_MAX, replace=False)
        umap_plot = umap_2d[idx]
    else:
        umap_plot = umap_2d

    FIGS_DIR.mkdir(parents=True, exist_ok=True)

    plt.figure(figsize=(8, 8), dpi=200)
    plt.scatter(umap_plot[:, 0], umap_plot[:, 1], s=1, alpha=0.5, linewidths=0, rasterized=True)
    plt.title("UMAP of Sessions (2D projection)")
    plt.xlabel("UMAP-1"); plt.ylabel("UMAP-2")
    plt.tight_layout()
    out_path = FIGS_DIR / f"{RUN_ID}_umap_sessions_2d.png"
    plt.savefig(out_path, dpi=200, bbox_inches="tight")
    plt.close()

    print("UMAP plot saved to:", out_path)


PCA | components: 96 | total explained variance: 0.806
k-means/HDBSCAN input shape: (371095, 96)
GMM input shape: (371095, 96)
DBSCAN sample size: 90000
HDBSCAN sample size: 90000


C:\ProgramData\anaconda3\envs\seg\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP -> (371095, 2)
Note: UMAP is for visual inspection only; global distances/densities are not faithful.
UMAP plot saved to: C:\Users\Admin\Documents\WBS\Dissertation\Submission Related files\Notebooks\Modelling\outputs\figs\20250827_144050_umap_sessions_2d.png
time: 9min 24s (started: 2025-08-27 15:07:07 +01:00)


## 4. Clustering (multiple unsupervised clustering models compared)

In [5]:
# 4) Clustering (KMeans, GMM[BIC], HDBSCAN + propagation, Agglomerative + propagation, DBSCAN + eps tuning)

from sklearn.cluster import MiniBatchKMeans, AgglomerativeClustering, DBSCAN as SK_DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier, kneighbors_graph
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan
import numpy as np
import pandas as pd
import time, os, warnings

warnings.filterwarnings("ignore", category=FutureWarning)

def tic(section):
    print(f"\n>>> Starting: {section}")
    return time.perf_counter()

def toc(t0, section):
    dt = time.perf_counter() - t0
    print(f"completed: {section} in {dt:.2f}s")

#  Shared evaluation sample (deterministic)
rng = np.random.default_rng(RANDOM_SEED)
if EVAL_ON_SAMPLE:
    EVAL_IDX = rng.choice(X_model_kmeans.shape[0], size=min(EVAL_SAMPLE_SIZE, X_model_kmeans.shape[0]), replace=False)
else:
    EVAL_IDX = np.arange(X_model_kmeans.shape[0])

def _eval_scores(X, y, metric="euclidean"):
    """Internal scores on the shared evaluation sample; ignores noise (-1)."""
    y_eval = y[EVAL_IDX]
    mask = (y_eval != NOISE_LABEL) if (NOISE_LABEL in y_eval) else np.ones_like(y_eval, bool)
    if mask.sum() < 50 or len(np.unique(y_eval[mask])) < 2:
        return {"sil": np.nan, "ch": np.nan, "db": np.nan}
    Xs, ys = X[EVAL_IDX][mask], y_eval[mask]
    out = {"sil": np.nan, "ch": np.nan, "db": np.nan}
    if COMPUTE_SILHOUETTE:
        out["sil"] = float(silhouette_score(Xs, ys, metric=metric))
    if COMPUTE_CH_DB:
        out["ch"]  = float(calinski_harabasz_score(Xs, ys))
        out["db"]  = float(davies_bouldin_score(Xs, ys))
    return out

labels_dict = {}
score_rows  = []

#  1) MiniBatch K-Means (on cosine-like L2 space)
t0 = tic("MiniBatch K-Means")
best_km, best_k, best_sil = None, None, -np.inf
for k in KMEANS_K_GRID:
    km = MiniBatchKMeans(
        n_clusters=k,
        batch_size=KMEANS_BATCH_SIZE,
        max_iter=KMEANS_MAX_ITER,
        n_init=KMEANS_N_INIT,
        random_state=RANDOM_SEED,
        init="k-means++"
    )
    km.fit(X_model_kmeans)
    labs = km.predict(X_model_kmeans)
    labels_dict[f"kmeans_{k}"] = labs
    s = _eval_scores(X_model_kmeans, labs)["sil"]
    if np.nan_to_num(s, nan=-1) > best_sil:
        best_sil, best_km, best_k = s, km, k
print("K-Means tried:", KMEANS_K_GRID, "| best k:", best_k, "| silhouette(eval):", None if np.isnan(best_sil) else round(best_sil, 4))
toc(t0, "MiniBatch K-Means")

#  2) GMM (choose K by BIC on PCA space)
t0 = tic("Gaussian Mixture Models (BIC)")
best_gmm, best_bic, best_g = None, np.inf, None
for g in GMM_K_GRID:
    gmm = GaussianMixture(
        n_components=g,
        covariance_type=GMM_COV_TYPE,
        random_state=GMM_RANDOM_STATE,
        max_iter=GMM_MAX_ITER,
        n_init=GMM_N_INIT,
        reg_covar=1e-6,
        init_params="kmeans"
    )
    gmm.fit(X_model_gmm)
    bic = gmm.bic(X_model_gmm)
    labels_dict[f"gmm_{g}"] = gmm.predict(X_model_gmm)
    if bic < best_bic:
        best_bic, best_g, best_gmm = bic, g, gmm
print("GMM best k by BIC:", best_g, "| BIC:", round(best_bic, 2))
toc(t0, "Gaussian Mixture Models (BIC)")

# 3) HDBSCAN (fit on sample if configured) + approximate_predict
t0 = tic("HDBSCAN + propagation")
n = X_model_hdbscan.shape[0]
fit_n = min(HDBSCAN_SAMPLE_SIZE, n) if USE_SAMPLED_CLUSTERING else n
fit_idx = (rng.choice(n, size=fit_n, replace=False) if fit_n < n else np.arange(n))

print(f"[HDBSCAN] fit_size={fit_idx.size:,}/{n:,} | min_cluster_size={HDBSCAN_MIN_CLUSTER_SIZE} | min_samples={HDBSCAN_MIN_SAMPLES} | metric={HDBSCAN_METRIC} | selection={HDBSCAN_CLUSTER_SELECTION}")
hdb = hdbscan.HDBSCAN(
    min_cluster_size=HDBSCAN_MIN_CLUSTER_SIZE,
    min_samples=HDBSCAN_MIN_SAMPLES,
    metric=HDBSCAN_METRIC,
    cluster_selection_method=HDBSCAN_CLUSTER_SELECTION,
    cluster_selection_epsilon=HDBSCAN_SELECTION_EPSILON,
    allow_single_cluster=HDBSCAN_ALLOW_SINGLE_CLUSTER,
    core_dist_n_jobs=(os.cpu_count() or 1),
    prediction_data=True
).fit(X_model_hdbscan[fit_idx])

from hdbscan import prediction as hdb_pred
if fit_idx.size < n and HDBSCAN_APPROX_PREDICT:
    labs_full, strengths = hdb_pred.approximate_predict(hdb, X_model_hdbscan)
    labs_full = labs_full.astype(int)
else:
    labs_full = hdb.labels_.astype(int)
    strengths = getattr(hdb, "probabilities_", np.ones_like(labs_full, dtype=float))

# optional confidence thresholding
if HDBSCAN_CLUSTER_PROB_THRESH > 0.0:
    low_conf = strengths < HDBSCAN_CLUSTER_PROB_THRESH
    labs_full = labs_full.copy()
    labs_full[low_conf] = NOISE_LABEL

labels_dict["hdbscan"] = labs_full
labels_dict["hdbscan_conf"] = strengths
hdb_scores = _eval_scores(X_model_hdbscan, labs_full, metric=("cosine" if HDBSCAN_METRIC == "cosine" else "euclidean"))
print("HDBSCAN clusters (excl. noise):", len(np.unique(labs_full[labs_full != NOISE_LABEL])))
print("HDBSCAN eval:", hdb_scores)
toc(t0, "HDBSCAN + propagation")

# 4) Agglomerative (Ward) + kNN propagation (kept as in prior runs)
if USE_AGGLO:
    t0 = tic("Agglomerative (Ward) + kNN propagation")
    # Fit on sample if sampling enabled
    agg_fit_n = min( max(AGGLOMERATIVE_NCL * 4_000, 60_000), X_model_kmeans.shape[0]) if USE_SAMPLED_CLUSTERING else X_model_kmeans.shape[0]
    agg_idx = (rng.choice(X_model_kmeans.shape[0], size=agg_fit_n, replace=False)
               if agg_fit_n < X_model_kmeans.shape[0] else np.arange(X_model_kmeans.shape[0]))
    X_agg_fit = X_model_kmeans[agg_idx]

    print(f"[Agglo] building connectivity graph (n={X_agg_fit.shape[0]:,}, k={K_CONNECTIVITY})")
    graph_conn = kneighbors_graph(X_agg_fit, n_neighbors=K_CONNECTIVITY, mode="connectivity", metric="euclidean")

    print(f"[Agglo] fitting Ward linkage with n_clusters={AGGLOMERATIVE_NCL}")
    agg_ward = AgglomerativeClustering(n_clusters=AGGLOMERATIVE_NCL, linkage="ward", connectivity=graph_conn)
    labels_agg_fit = agg_ward.fit_predict(X_agg_fit)

    print("Agglomerative: propagating labels to full dataset via kNN")
    knn_ag = KNeighborsClassifier(n_neighbors=K_PROPAGATE, weights="distance")
    knn_ag.fit(X_agg_fit, labels_agg_fit)
    labels_agg_full = knn_ag.predict(X_model_kmeans).astype(int)
    labels_dict[f"agg_ward_{AGGLOMERATIVE_NCL}"] = labels_agg_full

    agg_scores = _eval_scores(X_model_kmeans, labels_agg_full, metric="euclidean")
    print("Agglomerative clusters:", len(np.unique(labels_agg_full)))
    print("Agglomerative eval:", agg_scores)
    toc(t0, "Agglomerative (Ward) + kNN propagation")

# 5) DBSCAN (k-distance eps estimation on sample) + optional propagation
if USE_DBSCAN:
    t0 = tic("DBSCAN (eps via k-distance) + optional propagation")

    # choose sample for eps estimation (and for fit if sampling)
    eps_idx = (rng.choice(X_model_dbscan.shape[0], size=min(DBSCAN_EPS_SAMPLE_N, X_model_dbscan.shape[0]), replace=False)
               if USE_SAMPLED_CLUSTERING else np.arange(X_model_dbscan.shape[0]))
    X_db_eps = X_model_dbscan[eps_idx]

    print(f"[DBSCAN] estimating eps via k-distance on {X_db_eps.shape[0]:,} points (k={DBSCAN_MIN_SAMPLES}, metric={DBSCAN_METRIC})")
    nn = NearestNeighbors(n_neighbors=DBSCAN_MIN_SAMPLES, metric=DBSCAN_METRIC).fit(X_db_eps)
    dists, _ = nn.kneighbors(X_db_eps)
    kdist = np.sort(dists[:, -1])
    eps_grid = [float(np.percentile(kdist, p)) for p in [95, 96, 97, 98, 99]]

    best_eps, best_s = None, -np.inf
    for eps in eps_grid:
        tmp_labels = SK_DBSCAN(eps=eps, min_samples=DBSCAN_MIN_SAMPLES, metric=DBSCAN_METRIC).fit_predict(X_db_eps)
        # evaluate fairly on same eval index — propagate if sample != full
        if X_db_eps.shape[0] == X_model_dbscan.shape[0]:
            s = _eval_scores(X_model_dbscan, tmp_labels, metric=("cosine" if DBSCAN_METRIC == "cosine" else "euclidean"))["sil"]
        else:
            mask_fit = (tmp_labels != NOISE_LABEL)
            if mask_fit.sum() > 0:
                knn_db = KNeighborsClassifier(n_neighbors=K_PROPAGATE, weights="distance").fit(X_db_eps[mask_fit], tmp_labels[mask_fit])
                lab_full_tmp = np.full(X_model_dbscan.shape[0], NOISE_LABEL, dtype=int)
                lab_full_tmp[eps_idx] = tmp_labels
                rest_idx = np.setdiff1d(np.arange(X_model_dbscan.shape[0]), eps_idx, assume_unique=True)
                lab_full_tmp[rest_idx] = knn_db.predict(X_model_dbscan[rest_idx])
                s = _eval_scores(X_model_dbscan, lab_full_tmp, metric=("cosine" if DBSCAN_METRIC == "cosine" else "euclidean"))["sil"]
            else:
                s = -np.inf
        if np.nan_to_num(s, nan=-1) > best_s:
            best_s, best_eps = s, eps

    print("DBSCAN chosen eps:", round(best_eps, 5), "| silhouette(eval):", None if np.isnan(best_s) else round(best_s, 4))

    # final fit (sample + propagate, or full)
    if USE_SAMPLED_CLUSTERING:
        fit_idx = (rng.choice(X_model_dbscan.shape[0], size=min(DBSCAN_FIT_SAMPLE_N, X_model_dbscan.shape[0]), replace=False))
        X_db_fit = X_model_dbscan[fit_idx]
        db = SK_DBSCAN(eps=best_eps, min_samples=DBSCAN_MIN_SAMPLES, metric=DBSCAN_METRIC).fit(X_db_fit)
        lab_fit = db.labels_.astype(int)

        print("DBSCAN: propagating labels to full dataset via kNN...")
        labels_db_full = np.full(X_model_dbscan.shape[0], NOISE_LABEL, dtype=int)
        labels_db_full[fit_idx] = lab_fit
        mask_fit = (lab_fit != NOISE_LABEL)
        if mask_fit.sum() > 0:
            knn = KNeighborsClassifier(n_neighbors=K_PROPAGATE, weights="distance").fit(X_db_fit[mask_fit], lab_fit[mask_fit])
            rest_idx = np.setdiff1d(np.arange(X_model_dbscan.shape[0]), fit_idx, assume_unique=True)
            labels_db_full[rest_idx] = knn.predict(X_model_dbscan[rest_idx])
    else:
        db = SK_DBSCAN(eps=best_eps, min_samples=DBSCAN_MIN_SAMPLES, metric=DBSCAN_METRIC).fit(X_model_dbscan)
        labels_db_full = db.labels_.astype(int)

    db_key = f"dbscan_eps{round(best_eps,5)}_ms{DBSCAN_MIN_SAMPLES}"
    labels_dict[db_key] = labels_db_full
    db_scores = _eval_scores(X_model_dbscan, labels_db_full, metric=("cosine" if DBSCAN_METRIC == "cosine" else "euclidean"))
    print("DBSCAN clusters (excl. noise):", len(np.unique(labels_db_full[labels_db_full != NOISE_LABEL])))
    print("DBSCAN eval:", db_scores)
    toc(t0, "DBSCAN (eps via k-distance) + optional propagation")

# 6) Internal score summary + save
t0 = tic("Score summary + save")

def _n_clusters(y):
    u = np.unique(y)
    return int(len(u) - (1 if NOISE_LABEL in u else 0))

rows = []
rows.append(("kmeans_" + str(best_k),) + tuple(_eval_scores(X_model_kmeans, labels_dict[f"kmeans_{best_k}"]).values()))
rows.append(("gmm_" + str(best_g),) + tuple(_eval_scores(X_model_gmm,    labels_dict[f"gmm_{best_g}"]).values()))
rows.append(("hdbscan",)            + tuple(_eval_scores(X_model_hdbscan, labels_dict["hdbscan"], metric=("cosine" if HDBSCAN_METRIC=="cosine" else "euclidean")).values()))
if USE_AGGLO:
    rows.append((f"agg_ward_{AGGLOMERATIVE_NCL}",) + tuple(_eval_scores(X_model_kmeans, labels_dict[f"agg_ward_{AGGLOMERATIVE_NCL}"]).values()))
if USE_DBSCAN:
    rows.append((db_key,) + tuple(_eval_scores(X_model_dbscan, labels_dict[db_key], metric=("cosine" if DBSCAN_METRIC=="cosine" else "euclidean")).values()))

score_df = pd.DataFrame(rows, columns=["model", "silhouette", "ch", "db"])
# add #clusters and noise%
def _noise_pct(name):
    if name == "hdbscan":
        return float((labels_dict["hdbscan"] == NOISE_LABEL).mean())
    if name.startswith("dbscan_"):
        return float((labels_dict[name] == NOISE_LABEL).mean())
    return 0.0

score_df["n_clusters"] = score_df["model"].map(lambda m: _n_clusters(labels_dict["hdbscan"]) if m=="hdbscan"
                                               else (_n_clusters(labels_dict[db_key]) if m.startswith("dbscan_")
                                                     else _n_clusters(labels_dict[m])))
score_df["noise_pct"]  = score_df["model"].map(_noise_pct)

score_df.to_csv(SCORECARD_OUT, index=False)
print("Saved scorecard ->", SCORECARD_OUT)
print(score_df)
toc(t0, "Score summary + save")

# 7) Save full labels (all models)
t0 = tic("Saving labels (full)")
labels_df = pd.DataFrame({"session_id": session_to_items.index.to_numpy()})
for name, lab in labels_dict.items():
    if name.endswith("_conf"):
        continue
    labels_df[name] = lab
if "hdbscan_conf" in labels_dict:
    labels_df["hdbscan_conf"] = labels_dict["hdbscan_conf"]

LABELS_OUT = run_stem("cluster_labels_all_models.csv")
labels_df.to_csv(LABELS_OUT, index=False)
print("Saved labels ->", LABELS_OUT)
print("Clusterers saved:", list(labels_dict.keys()))
toc(t0, "Saving labels (full)")



>>> Starting: MiniBatch K-Means
K-Means tried: [2, 3, 4, 6, 8, 10, 12, 14] | best k: 2 | silhouette(eval): 0.097
completed: MiniBatch K-Means in 657.88s

>>> Starting: Gaussian Mixture Models (BIC)
GMM best k by BIC: 16 | BIC: 101105230.08
completed: Gaussian Mixture Models (BIC) in 212.63s

>>> Starting: HDBSCAN + propagation
[HDBSCAN] fit_size=90,000/371,095 | min_cluster_size=900 | min_samples=20 | metric=euclidean | selection=eom
HDBSCAN clusters (excl. noise): 2
HDBSCAN eval: {'sil': 0.1318977028131485, 'ch': 755.7602246410472, 'db': 1.726007524586368}
completed: HDBSCAN + propagation in 9160.75s

>>> Starting: Agglomerative (Ward) + kNN propagation
[Agglo] building connectivity graph (n=60,000, k=30)
[Agglo] fitting Ward linkage with n_clusters=12
Agglomerative: propagating labels to full dataset via kNN
Agglomerative clusters: 12
Agglomerative eval: {'sil': 0.06385967135429382, 'ch': 1529.730591965463, 'db': 3.6492577022254653}
completed: Agglomerative (Ward) + kNN propagation 

## 5. Baseline Static Model Implementation RFM Inspired

### 5.1. Session level RFM-style feature extraction


In [6]:
# 5.1) Feature extraction for session-level RFM proxies

# 1) Imports
import time, numpy as np, pandas as pd, math

def tic(msg):
    print(f">>> {msg}"); return time.perf_counter()
def toc(t0, msg):
    print(f"---- {msg} in {time.perf_counter()-t0:.2f}s")

assert {"session_id","timestamp","event","itemid"}.issubset(events.columns)

t0_all = tic("Session feature extraction (fast path)")

# 2) dtypes and normalisation (optimises groupby performance)
t0 = tic("cast dtypes")
ev = events[["session_id","timestamp","event","itemid"]].copy()
ev["event"] = ev["event"].astype(str).str.lower().astype("category")
if ev["itemid"].dtype != "category":
    ev["itemid"] = ev["itemid"].astype("category")
# respect global timestamp unit (supports ms->s if TS_DIV=1000)
ev["timestamp"] = (ev["timestamp"].astype("int64") // TS_DIV).astype("int64")
toc(t0, "cast dtypes")

# 3) Vectorised counts per (session,event)
t0 = tic("event counts (crosstab)")
counts = (ev.groupby(["session_id","event"], observed=True)
            .size()
            .unstack(fill_value=0))
# ensure expected columns exist (singular names to match PROFILE_FEATURES)
for col in ["view","addtocart","transaction"]:
    if col not in counts.columns:
        counts[col] = 0
# keep singular column names
counts = counts[["view","addtocart","transaction"]].astype("int32")
counts["events"] = counts.sum(axis=1).astype("int32")
toc(t0, "event counts done")

# 4) Timing statistics (min/max/duration) without sorting
t0 = tic("timings (min/max/duration)")
g = ev.groupby("session_id", observed=True, sort=False)
t_min = g["timestamp"].min()
t_max = g["timestamp"].max()
duration_sec = (t_max - t_min).astype("float64").clip(lower=0.0)
toc(t0, "timings done")

# 5) Unique items per session (item_diversity)
t0 = tic("unique items (item_diversity)")
item_diversity = g["itemid"].nunique().astype("int32")
toc(t0, "unique items done")

# 6) Last event per session -> one-hot
t0 = tic("last event via idxmax")
idx_last = g["timestamp"].idxmax()
last_event = ev.loc[idx_last, ["session_id","event"]].set_index("session_id")["event"]
last_dummies = pd.get_dummies(last_event, prefix="last_evt", dtype="int8")
for col in ["last_evt_view","last_evt_addtocart","last_evt_transaction"]:
    if col not in last_dummies.columns:
        last_dummies[col] = 0
toc(t0, "last event done")

# 7) Assemble features (rates, time-of-day, recency, bounce)
t0 = tic("assembling features")
sess = pd.DataFrame({
    "duration_sec": duration_sec,
    "item_diversity": item_diversity,
}).join(counts, how="left").join(last_dummies, how="left")

# rates/ratios
sess["events_per_min"] = (sess["events"] / np.maximum(sess["duration_sec"]/60.0, 1e-6)).astype("float64")

# time-of-day + day-of-week from session start
hour = pd.to_datetime(t_min, unit="s", errors="coerce").dt.hour.fillna(0).astype("int16")
dow  = pd.to_datetime(t_min, unit="s", errors="coerce").dt.dayofweek.fillna(0).astype("int8")
sess["hour"] = hour
sess["dow"]  = dow
sess["is_evening"] = ((hour >= 18) & (hour <= 23)).astype("int8")

# recency (days from dataset end to session end)
dataset_end = ev["timestamp"].max()
sess["recency_days"] = ((dataset_end - t_max) / (60*60*24)).astype("float64")

# bounce flag
sess["is_bounce"] = (sess["events"] <= 2).astype("int8")

# clean index name
sess.index.name = "session_id"
toc(t0, "features assembled")
toc(t0_all, "Session feature extraction")

# persist features for downstream SHAP/personas/reporting
out_feat = run_stem("session_features_rfm_proxies.csv")
sess.to_csv(out_feat)
print("Session-level feature table saved to:", out_feat)


>>> Session feature extraction (fast path)
>>> cast dtypes
---- cast dtypes in 0.94s
>>> event counts (crosstab)
---- event counts done in 0.35s
>>> timings (min/max/duration)
---- timings done in 0.08s
>>> unique items (item_diversity)
---- unique items done in 0.09s
>>> last event via idxmax
---- last event done in 0.09s
>>> assembling features
---- features assembled in 0.19s
---- Session feature extraction in 1.73s
Session-level feature table saved to: C:\Users\Admin\Documents\WBS\Dissertation\Submission Related files\Notebooks\Modelling\outputs\20250827_144050_session_features_rfm_proxies.csv
time: 4.66 s (started: 2025-08-27 18:18:37 +01:00)


### 5.2. Session-level RFM-proxy model (K-Means & profiling)


In [7]:
# 5.2) Session-level RFM-proxy model (K-Means & profiling)


# 1) Import
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler, RobustScaler
import time

# pulling from config
RANDOM_SEED  = globals().get("RANDOM_SEED", 42)
K_GRID_BASE  = globals().get("K_GRID_BASE", [2, 4, 6, 8, 10, 12])

# tiny timers
def tic(msg): print(f"\n>>> {msg}"); return time.perf_counter()
def toc(t0, msg): print(f"---- {msg} in {time.perf_counter()-t0:.2f}s")

# Preconditions
t0 = tic("RFM-proxy model — data preprocessing checks")
assert "sess" in globals(), "Expected 'sess' from the fast extractor."
need = {"recency_days","events","addtocart","transaction"}   # singular names per extractor
missing = need - set(sess.columns)
assert not missing, f"'sess' is missing columns: {missing}"
toc(t0, "pre-checks complete")

# 2) Construct R/F/M proxies
t0 = tic("Constructing R/F/M proxies")

R_proxy = sess["recency_days"].astype("float64")
F_proxy = np.log1p(sess["events"].astype("float64"))

tx   = sess["transaction"].astype("float64")
atc  = sess["addtocart"].astype("float64")
views = (sess["events"].astype("float64") - atc - tx).clip(lower=0)  # crude proxy

print(f"Non-zero rates -> tx: {(tx>0).mean():.4%}, atc: {(atc>0).mean():.4%}")

M_candidates = {
    "M_tx_plus_atc_log1p": np.log1p(tx + ATC_WEIGHT * atc),         # recommended
    "M_tx_log1p":          np.log1p(tx),                             # strict monetary
    "M_intent_per_view":   np.log1p((tx + ATC_WEIGHT * atc) / (1.0 + views))
}

chosen_M_name, M_proxy = None, None
for name, arr in M_candidates.items():
    if np.nanstd(arr) > 0:
        chosen_M_name, M_proxy = name, arr
        break

if M_proxy is None:
    print("All M candidates are constant -> falling back to RF baseline.")
    rfm_proxy = pd.DataFrame({
        "R_recency_days": R_proxy,
        "F_events_log1p": F_proxy,
    }, index=sess.index)
else:
    print(f"Chosen M proxy: {chosen_M_name}")
    rfm_proxy = pd.DataFrame({
        "R_recency_days": R_proxy,
        "F_events_log1p": F_proxy,
        "M_intent_log1p": M_proxy,
    }, index=sess.index)

# Scaling
USE_ROBUST = True  # set False to use StandardScaler
scaler = RobustScaler(quantile_range=(5, 95)) if USE_ROBUST else StandardScaler()
X_rfm = scaler.fit_transform(rfm_proxy.values).astype("float32", copy=False)

print("RFM-proxy preview:")
print(rfm_proxy.head().to_string())
toc(t0, "R/F/M proxies ready")

# 3) K-Means with sampled silhouette over the K grid
t0 = tic("Clustering RFM-proxy with K-Means + silhouette(K grid)")
n = X_rfm.shape[0]
rng = np.random.default_rng(RANDOM_SEED)
idx_sil = rng.choice(n, size=min(SIL_SAMPLE_N, n), replace=False)

best_k, best_sil, best_labels = None, -1.0, None
labels_by_k = {}

for k in K_GRID_BASE:
    if k < 2 or k >= n:
        continue
    km = KMeans(n_clusters=k, n_init=10, random_state=RANDOM_SEED)
    y  = km.fit_predict(X_rfm)
    labels_by_k[k] = y
    sil = silhouette_score(X_rfm[idx_sil], y[idx_sil], metric="euclidean")
    print(f"k={k:<2d} -> silhouette(sample)={sil:.4f}")
    if np.isfinite(sil) and sil > best_sil:
        best_k, best_sil, best_labels = k, sil, y

if best_k is None:
    raise RuntimeError("No valid K found. Please adjust K_GRID_BASE.")

print(f"Chosen K = {best_k} (silhouette={best_sil:.4f})")
toc(t0, "K-Means clustering completed for RFM Baseline model")

# 4) Profiling
t0 = tic("Profiling clusters (size, share, quantiles, intent rate)")
full_sil = float(silhouette_score(X_rfm, best_labels, metric="euclidean"))
print(f"[RFM-proxy] Full-data silhouette at K={best_k}: {full_sil:.4f}")

prof = rfm_proxy.copy()
prof["label"] = best_labels
prof["tx"]    = (sess["transaction"] > 0).astype(int)
prof["atc"]   = (sess["addtocart"] > 0).astype(int)

g = prof.groupby("label", observed=True)
cluster_size = g.size().rename("n")
share = (cluster_size / len(prof)).rename("share")
tx_rate  = g["tx"].mean().rename("tx_rate")
atc_rate = g["atc"].mean().rename("atc_rate")

def q(x):
    return pd.Series({"p25": np.percentile(x,25), "median": np.median(x), "p75": np.percentile(x,75)})

desc_parts = [g["R_recency_days"].apply(q).unstack().add_prefix("R_"),
              g["F_events_log1p"].apply(q).unstack().add_prefix("F_")]
if "M_intent_log1p" in prof.columns:
    desc_parts.append(g["M_intent_log1p"].apply(q).unstack().add_prefix("M_"))

desc = pd.concat(desc_parts, axis=1)
summary = pd.concat([cluster_size, share, tx_rate, atc_rate], axis=1).join(desc)
print(summary.round(3).to_string())
toc(t0, "profiling completed")

# 5) Saving results
t0 = tic("Saving results")

out = sess.join(rfm_proxy)
out["rfm_proxy_k"] = best_k
out["rfm_proxy_label"] = best_labels
out["rfm_full_silhouette"] = full_sil
out["rfm_chosen_M"] = chosen_M_name if chosen_M_name is not None else "none"
out["rfm_scaler"] = "Robust(5-95)" if isinstance(scaler, RobustScaler) else "Standard"

feat_path = run_stem("session_rfm_proxy_features.parquet")
lab_path  = run_stem(f"session_rfm_proxy_labels_k{best_k}.csv")
prof_path = run_stem(f"session_rfm_proxy_profile_k{best_k}.csv")

out.to_parquet(feat_path, index=True)
summary.reset_index().to_csv(prof_path, index=False)
pd.DataFrame({"session_id": out.index, "label": best_labels}).to_csv(lab_path, index=False)

print(f"Saved features -> {feat_path}")
print(f"Saved labels  -> {lab_path}")
print(f"Saved profile -> {prof_path}")

# expose labels for downstream agreement/SHAP steps
if "labels_dict" not in globals():
    labels_dict = {}
labels_dict[f"rfm_proxy_kmeans_session_{best_k}"] = best_labels

toc(t0, "results saved")



>>> RFM-proxy model — data preprocessing checks
---- pre-checks complete in 0.00s

>>> Constructing R/F/M proxies
Non-zero rates -> tx: 3.1609%, atc: 10.0451%
Chosen M proxy: M_tx_plus_atc_log1p
RFM-proxy preview:
            R_recency_days  F_events_log1p  M_intent_log1p
session_id                                                
1                 0.006262        1.386294             0.0
3                 0.041366        2.197225             0.0
8                 0.018843        1.609438             0.0
10                0.126887        1.098612             0.0
17                0.091933        1.098612             0.0
---- R/F/M proxies ready in 0.09s

>>> Clustering RFM-proxy with K-Means + silhouette(K grid)
k=4  -> silhouette(sample)=0.4175
k=6  -> silhouette(sample)=0.4656
k=8  -> silhouette(sample)=0.4175
k=10 -> silhouette(sample)=0.4405
k=12 -> silhouette(sample)=0.4465
Chosen K = 6 (silhouette=0.4656)
---- K-Means clustering completed for RFM Baseline model in 23.37s

>>> Pro

## 6. Visualising clusters across all algorithms (using UMAP 2D overlays)

In [8]:
# 6) Visualising clusters across all algorithms (UMAP overlays)


# 1) Imports
import numpy as np
import pandas as pd
import umap
import matplotlib.pyplot as plt
from pathlib import Path
import warnings, time, os

# 2) Progress helpers
def tic(msg): print(f"\n>>> {msg}"); return time.perf_counter()
def toc(t0, msg): print(f"---- {msg} in {time.perf_counter()-t0:.2f}s")

warnings.filterwarnings("ignore")

# 3) Preconditions & defaults
t0 = tic("UMAP overlays — prechecks")
assert "labels_dict" in globals() and isinstance(labels_dict, dict) and labels_dict, "labels_dict is missing/empty."

# pull-through from config
RANDOM_SEED = globals().get("RANDOM_SEED", 42)
RUN_ID      = globals().get("RUN_ID", "run")
UMAP_N_NEIGHBORS = int(globals().get("UMAP_N_NEIGHBORS", 30))
UMAP_MIN_DIST    = float(globals().get("UMAP_MIN_DIST", 0.05))
UMAP_METRIC      = globals().get("UMAP_METRIC", "cosine")

# figure/output dirs
FIGS_DIR = globals().get("FIGS_DIR", Path("outputs/figs"))
OUT_DIR  = globals().get("OUT_DIR", Path("outputs"))
FIGS_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)

# reproducible RNG
RNG = np.random.default_rng(RANDOM_SEED)

# Base neural-embedding matrix (prefer L2-PCA for cosine-like geometry)
if "X_model_kmeans" in globals():
    BASE_X = X_model_kmeans
elif "X_pca_l2" in globals():
    BASE_X = X_pca_l2
elif "X_pca" in globals():
    BASE_X = X_pca
else:
    raise AssertionError("Could not find BASE_X (X_model_kmeans/X_pca_l2/X_pca). Run the PCA block again (section 3).")
N = BASE_X.shape[0]

# Session index for alignment
if "SESSION_INDEX" in globals():
    SESSION_INDEX = SESSION_INDEX
elif "session_to_items" in globals():
    SESSION_INDEX = session_to_items.index.to_numpy()
else:
    SESSION_INDEX = (events[["session_id"]].drop_duplicates()
                     .astype({"session_id":"int64"}).values.ravel())

# Local plotting/runtime knobs
PLOT_MAX      = 100_000
POINT_SIZE    = 1.8
PLOT_DPI      = 140
REUSE_COORDS  = True
FIT_SAMPLE_N  = 120_000
BATCH_SIZE    = 100_000
LOW_MEMORY    = True
toc(t0, "prechecks completed")

# 4) Build or reuse UMAP for neural-embedding space
t0 = tic("UMAP base (neural-embedding space): sample-fit + transform")

coords_all = None
if REUSE_COORDS and ("umap_2d" in globals()) and (umap_2d is not None) and (getattr(umap_2d, "shape", (0,0))[0] == N):
    coords_all = umap_2d.astype(np.float32, copy=False)
    print("Reused precomputed umap_2d.")
else:
    fit_idx = RNG.choice(N, size=min(FIT_SAMPLE_N, N), replace=False)
    umap_model_all = umap.UMAP(
        n_neighbors=UMAP_N_NEIGHBORS,
        min_dist=UMAP_MIN_DIST,
        n_components=2,
        random_state=RANDOM_SEED,
        metric=("euclidean" if BASE_X is X_model_kmeans else UMAP_METRIC),  # L2-PCA -> euclidean ok
        low_memory=LOW_MEMORY,
    ).fit(BASE_X[fit_idx].astype(np.float32, copy=False))

    coords_all = np.empty((N, 2), dtype=np.float32)
    coords_all[fit_idx] = umap_model_all.embedding_.astype(np.float32, copy=False)

    rest = np.setdiff1d(np.arange(N), fit_idx, assume_unique=True)
    for s in range(0, len(rest), BATCH_SIZE):
        b = rest[s:s+BATCH_SIZE]
        coords_all[b] = umap_model_all.transform(BASE_X[b].astype(np.float32, copy=False)).astype(np.float32, copy=False)

    # expose for later reuse
    umap_2d = coords_all

# Save coordinates (run-stamped under outputs/)
coords_all_df = pd.DataFrame({
    "session_id": SESSION_INDEX,
    "umap_x": coords_all[:,0],
    "umap_y": coords_all[:,1],
})
coords_all_df.to_parquet(run_stem("umap_all_models_coords.parquet"), index=False)
coords_all_df.to_csv(run_stem("umap_all_models_coords.csv"), index=False)
print("Saved base UMAP coords to:", run_stem("umap_all_models_coords.parquet"), "and", run_stem("umap_all_models_coords.csv"))
print("Ethical Note: UMAP is for visual inspection only and not a clustering tool.")
toc(t0, "UMAP base ready")

# 5) Plot helper (centroids computed from plotted subset only)
def _plot_clusters(coords2d, labels, title, out_png, max_points=PLOT_MAX):
    n = coords2d.shape[0]
    if n > max_points:
        idx = RNG.choice(n, size=max_points, replace=False)
        pts, labs = coords2d[idx], labels[idx]
    else:
        pts, labs = coords2d, labels

    # color mapping: noise in light gray
    base_colors = plt.rcParams['axes.prop_cycle'].by_key().get('color', ['C0','C1','C2','C3','C4','C5','C6','C7','C8','C9'])
    uniq = np.unique(labs)
    uniq_sorted = [u for u in uniq if u != -1] + ([-1] if -1 in uniq else [])
    color_map, c_idx = {}, 0
    for u in uniq_sorted:
        if u == -1:
            color_map[u] = "#D3D3D3"
        else:
            color_map[u] = base_colors[c_idx % len(base_colors)]
            c_idx += 1

    plt.figure(figsize=(8,8), dpi=PLOT_DPI)
    for u in uniq_sorted:
        m = (labs == u)
        if not np.any(m): 
            continue
        plt.scatter(pts[m,0], pts[m,1], s=POINT_SIZE, alpha=0.55, linewidths=0,
                    color=color_map[u], label=("Noise" if u == -1 else f"Cluster {u}"), rasterized=True)

    # centroids from the plotted subset
    cents = []
    for u in [u for u in uniq_sorted if u != -1]:
        m = (labs == u)
        if np.any(m):
            cents.append(pts[m].mean(axis=0))
    if cents:
        cents = np.vstack(cents)
        plt.scatter(cents[:,0], cents[:,1], s=120, marker="X", edgecolor="black",
                    linewidths=0.5, color=[color_map[u] for u in uniq_sorted if u != -1], label="Centroids")

    plt.title(title)
    plt.xlabel("UMAP-1"); plt.ylabel("UMAP-2")
    plt.legend(markerscale=4, fontsize=8, frameon=True)
    plt.tight_layout()
    plt.savefig(out_png, dpi=PLOT_DPI, bbox_inches="tight")
    plt.close()
    print("Saved:", out_png)

# 6) Label alignment helper
def _align_labels(name, arr):
    """Ensures that labels align with BASE_X row order (SESSION_INDEX). Returns None if not alignable."""
    if len(arr) == len(SESSION_INDEX):
        return np.asarray(arr)
    if ("rfm_proxy" in globals()) and isinstance(rfm_proxy, pd.DataFrame):
        if len(arr) == rfm_proxy.shape[0]:
            lab_series = pd.Series(arr, index=rfm_proxy.index)
            aligned = lab_series.reindex(SESSION_INDEX).to_numpy()
            return aligned if aligned.shape[0] == len(SESSION_INDEX) else None
        if name == "rfm_proxy" and "rfm_proxy_label" in rfm_proxy.columns:
            aligned = rfm_proxy["rfm_proxy_label"].reindex(SESSION_INDEX).to_numpy()
            return aligned if aligned.shape[0] == len(SESSION_INDEX) else None
    return None

# 7) Plot overlays for ALL models on the neural embedding UMAP
t0 = tic("Plotting neural-embedding overlays for all models")
all_plots = []  # (title, filename)

for key, lab in labels_dict.items():
    if key.endswith("_conf"):  # skip HDBSCAN strengths
        continue
    labels_aligned = _align_labels(key, lab)
    if labels_aligned is None:
        print(f"[skip] {key} — label array length mismatch (expected {len(SESSION_INDEX)})")
        continue

    pretty = (key
              .replace("_knnprop_conf","")
              .replace("_knnprop","")
              .replace("agg_","Agglomerative ")
              .replace("kmeans_","KMeans k=")
              .replace("gmm_","GMM k=")
              .replace("dbscan_","DBSCAN ")
              .replace("hdbscan","HDBSCAN"))
    fname = f"{RUN_ID}_umap_{key}.png".replace("__","_")
    out_png = FIGS_DIR / fname
    _plot_clusters(coords_all, labels_aligned, f"Neural-embedding UMAP — {pretty}", out_png)
    all_plots.append((pretty, out_png))

toc(t0, "neural overlays completed")

# 8) Separate UMAP for RFM baseline (3 features)
if ("X_rfm" in globals()) and ("rfm_proxy" in globals()):
    t0 = tic("UMAP for session RFM-proxy (3 features): sample-fit + transform")
    X_rfm = X_rfm.astype(np.float32, copy=False)
    n_rfm  = X_rfm.shape[0]
    fit_n  = min(60_000, n_rfm)
    fit_idx = RNG.choice(n_rfm, size=fit_n, replace=False)

    umap_rfm = umap.UMAP(
        n_neighbors=UMAP_N_NEIGHBORS,
        min_dist=UMAP_MIN_DIST,
        n_components=2,
        random_state=RANDOM_SEED,
        metric="euclidean",
        low_memory=LOW_MEMORY,
    ).fit(X_rfm[fit_idx])

    coords_rfm = np.empty((n_rfm, 2), dtype=np.float32)
    coords_rfm[fit_idx] = umap_rfm.embedding_.astype(np.float32, copy=False)
    rest = np.setdiff1d(np.arange(n_rfm), fit_idx, assume_unique=True)
    for s in range(0, len(rest), BATCH_SIZE):
        b = rest[s:s+BATCH_SIZE]
        coords_rfm[b] = umap_rfm.transform(X_rfm[b]).astype(np.float32, copy=False)

    # plot any RFM labels present
    for key, lab in labels_dict.items():
        if key.startswith("rfm_proxy_") and (not key.endswith("_conf")):
            labels_rfm = _align_labels("rfm_proxy", lab)
            if labels_rfm is None:
                print(f"[skip RFM plot] {key} — label array length mismatch (expected {len(SESSION_INDEX)})")
                continue
            out_png = FIGS_DIR / f"{RUN_ID}_umap_{key}.png"
            _plot_clusters(coords_rfm, labels_rfm, f"RFM-proxy UMAP — {key}", out_png)
            all_plots.append((key, out_png))

    # saving RFM coordinates (run-stamped)
    coords_rfm_df = pd.DataFrame({
        "session_id": rfm_proxy.index.values,
        "umap_x": coords_rfm[:,0],
        "umap_y": coords_rfm[:,1],
    })
    coords_rfm_df.to_parquet(run_stem("umap_rfm_proxy_coords.parquet"), index=False)
    coords_rfm_df.to_csv(run_stem("umap_rfm_proxy_coords.csv"), index=False)
    toc(t0, "RFM-proxy UMAP completed")
else:
    print("Skipped RFM-proxy UMAP: X_rfm/rfm_proxy not found (run 5.1 & 5.2 first).")

print("\nGenerated plots:")
for ttitle, p in all_plots:
    print("-", ttitle, "->", p)



>>> UMAP overlays — prechecks
---- prechecks completed in 0.00s

>>> UMAP base (neural-embedding space): sample-fit + transform
Reused precomputed umap_2d.
Saved base UMAP coords to: C:\Users\Admin\Documents\WBS\Dissertation\Submission Related files\Notebooks\Modelling\outputs\20250827_144050_umap_all_models_coords.parquet and C:\Users\Admin\Documents\WBS\Dissertation\Submission Related files\Notebooks\Modelling\outputs\20250827_144050_umap_all_models_coords.csv
Note: UMAP is for visual inspection only and not a clustering tool.
---- UMAP base ready in 1.26s

>>> Plotting neural-embedding overlays for all models
Saved: C:\Users\Admin\Documents\WBS\Dissertation\Submission Related files\Notebooks\Modelling\outputs\figs\20250827_144050_umap_kmeans_2.png
Saved: C:\Users\Admin\Documents\WBS\Dissertation\Submission Related files\Notebooks\Modelling\outputs\figs\20250827_144050_umap_kmeans_3.png
Saved: C:\Users\Admin\Documents\WBS\Dissertation\Submission Related files\Notebooks\Modelling\out

## 7. Model evaluation (internal metrics, cross-model agreement and temporal drift)

In [9]:
# 7) Model evaluation (internal metrics, cross-model agreement, temporal drift)


# 1) Imports
import numpy as np
import pandas as pd
import time, os
from sklearn.metrics import (
    silhouette_score, calinski_harabasz_score, davies_bouldin_score,
    adjusted_rand_score, adjusted_mutual_info_score
)

# 2) small timers
def tic(section):
    print(f"\n>>> Starting: {section}")
    return time.perf_counter()

def toc(t0, section):
    dt = time.perf_counter() - t0
    print(f"---- completed: {section} in {dt:.2f}s")

# 3) Pre-checks & pulling through config
t0 = tic("Evaluation — prechecks")
assert "labels_dict" in globals() and len(labels_dict) > 0, "labels_dict not found; run clustering first."

RANDOM_SEED        = globals().get("RANDOM_SEED", 42)
RNG                = globals().get("RNG", np.random.default_rng(RANDOM_SEED))
EVAL_SAMPLE_SIZE   = int(globals().get("EVAL_SAMPLE_SIZE", 10_000))
COMPUTE_SILHOUETTE = bool(globals().get("COMPUTE_SILHOUETTE", True))
COMPUTE_CH_DB      = bool(globals().get("COMPUTE_CH_DB", True))
COMPUTE_DBCV       = bool(globals().get("COMPUTE_DBCV", False))
NOISE_LABEL        = int(globals().get("NOISE_LABEL", -1))

OUT_DIR.mkdir(parents=True, exist_ok=True)

# Reuse existing S_EVAL_IDX if present; else deterministic sample
if "S_EVAL_IDX" not in globals():
    N_ref = next(v.shape[0] for k, v in labels_dict.items() if not k.endswith("_conf"))
    S_EVAL_IDX = RNG.choice(N_ref, size=min(EVAL_SAMPLE_SIZE, N_ref), replace=False)
print(f"Eval sample size: {len(S_EVAL_IDX)}")
toc(t0, "Evaluation — prechecks completed")

# 4) Choose the feature space matching each model’s labels
def _X_for_label(key: str):
    """
    Return the feature matrix that matches how 'key' labels were produced.
    Falls back to X_model_kmeans (L2-PCA) if ambiguous.
    """
    if key.startswith("kmeans_") or key.startswith("agg_"):
        return X_model_kmeans
    if key.startswith("gmm_"):
        return X_model_gmm
    if key.startswith("hdbscan_"):
        return X_model_hdbscan
    if key.startswith("dbscan_"):
        return X_model_dbscan
    if key.startswith("rfm_simple") or key.startswith("rfm_proxy"):
        return globals().get("X_rfm", X_model_kmeans)
    return X_model_kmeans

# 5) Internal metrics per model (silhouette / CH / DB / optional DBCV)
t0 = tic("Internal metrics per model")

rows = []
for name, y_full in labels_dict.items():
    if name.endswith("_conf"):
        continue

    X = _X_for_label(name)
    idx = S_EVAL_IDX if len(S_EVAL_IDX) < X.shape[0] else np.arange(X.shape[0])

    y = y_full[idx]
    # ignore noise for geometry metrics
    mask = (y != NOISE_LABEL) if (NOISE_LABEL in y) else np.ones_like(y, dtype=bool)
    Xs, ys = X[idx][mask], y[mask]

    sil = ch = db = np.nan
    dbcv = np.nan

    # need at least 2 clusters & enough points
    if (np.unique(ys).size >= 2) and (Xs.shape[0] >= 100):
        # stratified subsample for silhouette (keep ≥2 labels)
        Xsub, ysub = Xs, ys
        sub_n = min(5_000, Xs.shape[0])
        if Xs.shape[0] > sub_n:
            labels_u = np.unique(ys)
            per = max(1, int(np.ceil(sub_n / labels_u.size)))
            take_idx = []
            for c in labels_u:
                idx_c = np.where(ys == c)[0]
                if idx_c.size > 0:
                    k = min(per, idx_c.size)
                    take_idx.append(RNG.choice(idx_c, size=k, replace=False))
            take_idx = np.concatenate(take_idx)
            if np.unique(ys[take_idx]).size >= 2:
                Xsub, ysub = Xs[take_idx], ys[take_idx]

        if COMPUTE_SILHOUETTE:
            sil = float(silhouette_score(Xsub, ysub, metric="euclidean"))
        if COMPUTE_CH_DB:
            ch  = float(calinski_harabasz_score(Xs, ys))
            db  = float(davies_bouldin_score(Xs, ys))

        # Optional: DBCV (density-based validity), best for density models
        if COMPUTE_DBCV:
            try:
                from hdbscan.validity import validity_index
                # Use cosine-like geometry when X is L2-normalised (kmeans/dbscan/hdbscan on L2-PCA)
                # validity_index expects condensed distances or a metric name supported by pairwise distances.
                # We’ll use 'euclidean' here because BASE_X is L2-normalised for cosine-like setups.
                dbcv = float(validity_index(Xs, y=ys, metric='euclidean'))
            except Exception as e:
                # keep robust; just log once per model
                print(f"[DBCV skipped for {name}] {e}")

    # count clusters excl. noise
    n_clusters = int(np.unique(y_full[y_full != NOISE_LABEL]).size) if (NOISE_LABEL in y_full) else int(np.unique(y_full).size)
    rows.append({
        "model": name, "n_clusters": n_clusters,
        "silhouette": sil, "calinski_harabasz": ch, "davies_bouldin": db,
        "dbcv": dbcv
    })

internal_df = pd.DataFrame(rows).sort_values(["silhouette"], ascending=False)
internal_path = run_stem("eval_internal_metrics.csv")
internal_df.to_csv(internal_path, index=False)
print("Saved internal metrics to:", internal_path)
print(internal_df.head(10))
toc(t0, "Internal metrics per model")

# 6) Cross-model agreement (ARI / AMI)
t0 = tic("Cross-model agreement (ARI / AMI)")

keys = [k for k in labels_dict.keys() if not k.endswith("_conf")]
Y = {k: labels_dict[k] for k in keys}

ari = pd.DataFrame(index=keys, columns=keys, dtype="float32")
ami = pd.DataFrame(index=keys, columns=keys, dtype="float32")

for ki in keys:
    yi = Y[ki]
    for kj in keys:
        yj = Y[kj]
        if len(yi) == len(yj):
            ari.loc[ki, kj] = adjusted_rand_score(yi, yj)
            ami.loc[ki, kj] = adjusted_mutual_info_score(yi, yj)
        else:
            ari.loc[ki, kj] = np.nan
            ami.loc[ki, kj] = np.nan

ari_path = run_stem("eval_cross_model_ARI.csv")
ami_path = run_stem("eval_cross_model_AMI.csv")
ari.to_csv(ari_path); ami.to_csv(ami_path)
print("Saved ARI ->", ari_path)
print("Saved AMI ->", ami_path)

def _top_pairs(M, k=5):
    vals = []
    for i in M.index:
        for j in M.columns:
            if i >= j:  # avoid dup/self (matrix is symmetric)
                continue
            v = M.loc[i, j]
            if pd.notnull(v):
                vals.append((i, j, float(v)))
    vals.sort(key=lambda x: x[2], reverse=True)
    return vals[:k], vals[-k:]

top5, bot5 = _top_pairs(ari)
print("\nTop 5 ARI pairs:")
for a,b,v in top5:  print(f"{a:>30s}  vs  {b:<30s}  ->  ARI={v:.3f}")
print("\nBottom-5 ARI pairs:")
for a,b,v in bot5:  print(f"{a:>30s}  vs  {b:<30s}  ->  ARI={v:.3f}")

toc(t0, "Cross-model agreement (ARI / AMI)")

# 7) Temporal drift (early vs late halves by session start time)
t0 = tic("Temporal drift check (early vs late halves)")

# Build/confirm SESSION_INDEX
if "SESSION_INDEX" not in globals():
    if "session_to_items" in globals():
        SESSION_INDEX = session_to_items.index.to_numpy()
    else:
        SESSION_INDEX = (events[["session_id"]].drop_duplicates()
                         .astype({"session_id":"int64"}).values.ravel())

# Respect TS_DIV from config
TS_DIV = int(globals().get("TS_DIV", 1))
sess_first_ts = (events.groupby("session_id")["timestamp"].min() // TS_DIV)
sess_first_ts = sess_first_ts.reindex(SESSION_INDEX).to_numpy()
median_ts = np.nanmedian(sess_first_ts)
early = (sess_first_ts <= median_ts)
late  = ~early

drift_rows = []
for name, y in Y.items():
    labs = np.asarray(y)
    uniq = np.unique(labs)
    for c in uniq:
        if c == NOISE_LABEL:  # skip noise here
            continue
        p_early = float((labs[early] == c).mean())
        p_late  = float((labs[late]  == c).mean())
        drift_rows.append({
            "model": name, "cluster": int(c),
            "p_early": p_early, "p_late": p_late,
            "abs_diff": abs(p_early - p_late)
        })

drift_df = pd.DataFrame(drift_rows).sort_values(["abs_diff"], ascending=False)
TEMPORAL_OUT = run_stem("eval_temporal_drift.csv")
drift_df.to_csv(TEMPORAL_OUT, index=False)
print("Saved temporal drift summary to:", TEMPORAL_OUT)
print(drift_df.head(10))
toc(t0, "Temporal drift check")



>>> Starting: Evaluation — prechecks
Eval sample size: 70000
---- completed: Evaluation — prechecks completed in 0.00s

>>> Starting: Internal metrics per model
[DBCV skipped for kmeans_2] validity_index() missing 1 required positional argument: 'labels'
[DBCV skipped for kmeans_3] validity_index() missing 1 required positional argument: 'labels'
[DBCV skipped for kmeans_4] validity_index() missing 1 required positional argument: 'labels'
[DBCV skipped for kmeans_6] validity_index() missing 1 required positional argument: 'labels'
[DBCV skipped for kmeans_8] validity_index() missing 1 required positional argument: 'labels'
[DBCV skipped for kmeans_10] validity_index() missing 1 required positional argument: 'labels'
[DBCV skipped for kmeans_12] validity_index() missing 1 required positional argument: 'labels'
[DBCV skipped for kmeans_14] validity_index() missing 1 required positional argument: 'labels'
[DBCV skipped for gmm_3] validity_index() missing 1 required positional argument: '

## 8. Explainability via a surrogate RandomForest + SHAP

In [10]:
# 7) Surrogate explainability with SHAP

# 1) Imports
import os, gc, joblib, numpy as np, pandas as pd, shap
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier

# 2) pulling through from config (defaults safe if missing)
USE_SHAP_SURROGATE   = bool(globals().get("USE_SHAP_SURROGATE", True))
if not USE_SHAP_SURROGATE:
    print("[SHAP] USE_SHAP_SURROGATE=False, skipping this section.")
else:
    RANDOM_SEED         = int(globals().get("RANDOM_SEED", 42))
    RNG                 = globals().get("RNG", np.random.default_rng(RANDOM_SEED))
    N_JOBS              = int(globals().get("N_JOBS", -1))
    NOISE_LABEL         = int(globals().get("NOISE_LABEL", -1))

    OUT_DIR             = globals().get("OUT_DIR", Path("outputs")); OUT_DIR.mkdir(parents=True, exist_ok=True)
    SHAP_DIR            = globals().get("SHAP_DIR", OUT_DIR / "shap_artifacts"); SHAP_DIR.mkdir(parents=True, exist_ok=True)

    RF_N_ESTIMATORS     = int(globals().get("RF_N_ESTIMATORS", 400))
    RF_MAX_DEPTH        = globals().get("RF_MAX_DEPTH", None)
    RF_RANDOM_STATE     = RANDOM_SEED
    SHAP_SAMPLE_SIZE    = int(globals().get("SHAP_SAMPLE_SIZE", 100_000))     # total cap across clusters
    SHAP_TOP_N_FEATURES = int(globals().get("SHAP_TOP_N_FEATURES", 15))
    SHAP_BACKGROUND_SIZE= int(globals().get("SHAP_BACKGROUND_SIZE", 2_000))
    SAVE_BEESWARM_PNG   = bool(globals().get("SAVE_BEESWARM_PNG", True))

    assert "labels_dict" in globals() and labels_dict, "labels_dict is missing. Run clustering first."

    # 3) Pick the lead clustering model (prefer neural over RFM unless LEAD_KEY provided)
    lead_key = globals().get("LEAD_KEY", None)

    def _pick_lead_key():
        if "internal_df" in globals() and isinstance(internal_df, pd.DataFrame) and not internal_df.empty:
            order = internal_df.sort_values("silhouette", ascending=False)["model"].tolist()
            for m in order:
                if not m.startswith("rfm_proxy_"):
                    return m
            return order[0]
        path = OUT_DIR / "eval_internal_metrics.csv"
        if path.exists():
            df = pd.read_csv(path)
            order = df.sort_values("silhouette", ascending=False)["model"].tolist()
            for m in order:
                if not m.startswith("rfm_proxy_"):
                    return m
            return order[0]
        # fallback: any kmeans_* else first non-conf entry
        return next((k for k in labels_dict if k.startswith("kmeans_") and not k.endswith("_conf")),
                    next(k for k in labels_dict if not k.endswith("_conf")))

    if lead_key is None:
        lead_key = _pick_lead_key()

    print(f"[SHAP] Lead model: {lead_key}")
    y_all = np.asarray(labels_dict[lead_key])

    # 4) Feature table for surrogate (use 'out' if present, else 'sess')
    feat_df = out.copy() if "out" in globals() else sess.copy()

    # Align row order to SESSION_INDEX (same as clustering inputs)
    if "SESSION_INDEX" in globals():
        if "session_id" in feat_df.columns:
            feat_df = feat_df.set_index("session_id")
        feat_df = feat_df.reindex(pd.Index(SESSION_INDEX))

    # Drop non-numeric + housekeeping columns, clean NaN/inf, remove constant cols
    drop_cols = [c for c in feat_df.columns if c.startswith("rfm_proxy_")] + ["session_id"]
    feat_df = (feat_df.drop(columns=drop_cols, errors="ignore")
                     .select_dtypes(include=[np.number])
                     .replace([np.inf, -np.inf], np.nan)
                     .fillna(0.0))
    feat_df = feat_df.loc[:, feat_df.nunique(dropna=False) > 1]

    X_full = feat_df.to_numpy().astype("float32", copy=False)
    y_full = y_all

    # Remove noise labels from training
    mask_train = (y_full != NOISE_LABEL)
    X_train, y_train = X_full[mask_train], y_full[mask_train]
    assert X_train.shape[0] > 0, "No non-noise labels available for SHAP surrogate."

    # 5) Train/load RandomForest surrogate (fully configurable)
    rf_path = SHAP_DIR / f"rf_surrogate_{lead_key}.joblib"
    if rf_path.exists():
        rf = joblib.load(rf_path)
        print("[SHAP] Loaded surrogate RF from disk.")
    else:
        rf = RandomForestClassifier(
            n_estimators=RF_N_ESTIMATORS,
            max_depth=RF_MAX_DEPTH,
            max_features="sqrt",
            min_samples_leaf=20,
            n_jobs=N_JOBS if isinstance(N_JOBS, int) else -1,
            random_state=RF_RANDOM_STATE
        ).fit(X_train, y_train)
        joblib.dump(rf, rf_path)
        print("[SHAP] Trained & saved surrogate RF.")

    # 6) Balanced, capped SHAP sample across clusters (total cap = SHAP_SAMPLE_SIZE)
    classes = np.sort(np.unique(y_train))
    K = len(classes)
    per_cls = max(500, int(np.ceil(SHAP_SAMPLE_SIZE / max(1, K))))
    take_idx = []
    for c in classes:
        idx_c = np.where(y_train == c)[0]
        if idx_c.size == 0:
            continue
        take = min(per_cls, idx_c.size)
        take_idx.append(RNG.choice(idx_c, size=take, replace=False))
    take_idx = np.concatenate(take_idx) if take_idx else np.arange(min(10_000, X_train.shape[0]))
    X_shap, y_shap = X_train[take_idx], y_train[take_idx]
    print(f"[SHAP] Sampled for SHAP: {X_shap.shape[0]} rows across {K} clusters.")
    pd.Series(y_shap).value_counts().sort_index().to_csv(SHAP_DIR / f"shap_sample_class_sizes_{lead_key}.csv")

    # 7) Background for TreeExplainer (interventional): small, stratified
    bg_per_cls = max(50, int(np.ceil(SHAP_BACKGROUND_SIZE / max(1, K))))
    bg_idx = []
    for c in classes:
        idx_c = np.where(y_train == c)[0]
        if idx_c.size == 0:
            continue
        take = min(bg_per_cls, idx_c.size)
        bg_idx.append(RNG.choice(idx_c, size=take, replace=False))
    bg_idx = np.concatenate(bg_idx) if bg_idx else RNG.choice(X_train.shape[0], size=min(SHAP_BACKGROUND_SIZE, X_train.shape[0]), replace=False)
    X_bg = X_train[bg_idx]

    # Helper to normalise SHAP outputs across versions
    def _as_class_list(sv):
        if isinstance(sv, list):
            return sv
        if isinstance(sv, np.ndarray):
            if sv.ndim == 3:               # (n,F,C)
                return [sv[:, :, i] for i in range(sv.shape[2])]
            if sv.ndim == 2:               # (n,F)
                return [sv]
        raise ValueError(f"Unexpected SHAP output shape/type: {type(sv)}, ndim={getattr(sv,'ndim',None)}")

    # 8) Streaming SHAP aggregation (memory friendly)
    explainer = shap.TreeExplainer(rf, data=X_bg, feature_perturbation="interventional", model_output="raw")
    BATCH = 2000
    F     = X_shap.shape[1]
    C     = len(rf.classes_)
    sum_abs_global     = np.zeros(F, dtype=np.float64)
    count_global       = 0
    sum_abs_by_cluster = np.zeros((C, F), dtype=np.float64)
    count_by_cluster   = np.zeros(C, dtype=np.int64)

    for s in range(0, X_shap.shape[0], BATCH):
        xb = X_shap[s:s+BATCH]; yb = y_shap[s:s+BATCH]
        raw = explainer.shap_values(xb, check_additivity=False)
        clist = _as_class_list(raw)  # list of (n_batch,F)

        # multiclass alignment; SHAP may return 1 array in binary case
        if len(clist) == 1 and C == 2:
            clist = [clist[0], clist[0]]

        for ci, c in enumerate(rf.classes_):
            sv_ci = clist[min(ci, len(clist)-1)]
            m = (yb == c)
            if not np.any(m):
                continue
            v = np.abs(sv_ci[m]).sum(axis=0)    # (F,)
            sum_abs_by_cluster[ci] += v
            count_by_cluster[ci]   += int(m.sum())
            sum_abs_global         += v
            count_global           += int(m.sum())

        del raw, clist, xb, yb
        if (s // BATCH) % 5 == 0:
            np.savez_compressed(
                SHAP_DIR / f"shap_ckpt_{lead_key}.npz",
                sum_abs_global=sum_abs_global, count_global=np.array([count_global]),
                sum_abs_by_cluster=sum_abs_by_cluster, count_by_cluster=count_by_cluster
            )
            gc.collect()

    # 9) Save summaries (timestamped via run_stem if available)
    feat_names = feat_df.columns.to_list()

    def _out(path_like):
        # use run_stem if available for consistent RUN_ID prefix
        return run_stem(path_like) if "run_stem" in globals() else (OUT_DIR / path_like)

    global_imp = pd.DataFrame({
        "feature": feat_names,
        "mean_abs_shap": sum_abs_global / max(1, count_global)
    }).sort_values("mean_abs_shap", ascending=False)
    global_path = _out(f"shap_global_{lead_key}.csv")
    global_imp.to_csv(global_path, index=False)

    per_cluster_rows = []
    for ci, c in enumerate(rf.classes_):
        if count_by_cluster[ci] == 0:
            continue
        tmp = pd.DataFrame({
            "cluster": int(c),
            "feature": feat_names,
            "mean_abs_shap": (sum_abs_by_cluster[ci] / max(1, count_by_cluster[ci]))
        }).sort_values("mean_abs_shap", ascending=False).head(SHAP_TOP_N_FEATURES)
        tmp["rank"] = np.arange(1, len(tmp) + 1)
        per_cluster_rows.append(tmp)

    top_path = _out(f"shap_top_features_per_cluster_{lead_key}.csv")
    if per_cluster_rows:
        pd.concat(per_cluster_rows, ignore_index=True).to_csv(top_path, index=False)

    # Optional extra export if SHAP_TABLE_OUT set in config
    SHAP_TABLE_OUT = globals().get("SHAP_TABLE_OUT", None)
    if SHAP_TABLE_OUT is not None and per_cluster_rows:
        pd.concat(per_cluster_rows, ignore_index=True).to_csv(SHAP_TABLE_OUT, index=False)

    print("[SHAP] Saved:", global_path, "and", top_path)

    # 10) Tiny beeswarm (subsampled) if enabled
    if SAVE_BEESWARM_PNG and X_shap.shape[0] > 0:
        import matplotlib.pyplot as plt
        idx = RNG.choice(X_shap.shape[0], size=min(3000, X_shap.shape[0]), replace=False)
        raw   = explainer.shap_values(X_shap[idx], check_additivity=False)
        clist = _as_class_list(raw)

        plt.figure(figsize=(8,6), dpi=140)
        if len(clist) > 1:  # multiclass or mirrored binary
            counts = [np.sum(y_shap[idx] == c) for c in rf.classes_]
            top_ci = int(np.argmax(counts))
            shap.summary_plot(clist[min(top_ci, len(clist)-1)], X_shap[idx],
                              feature_names=feat_names, show=False)
        else:
            shap.summary_plot(clist[0], X_shap[idx], feature_names=feat_names, show=False)
        plt.tight_layout()
        fig_path = _out(f"shap_beeswarm_{lead_key}.png")
        plt.savefig(fig_path, dpi=140, bbox_inches="tight")
        plt.close()
        print("[SHAP] Saved beeswarm:", fig_path)

    print("Ethical Note: SHAP indicates feature association with cluster assignments, not causality. Correlated features may share credit.")


[SHAP] Lead model: kmeans_2
[SHAP] Trained & saved surrogate RF.
[SHAP] Sampled for SHAP: 30000 rows across 2 clusters.


100%|===================| 3994/4000 [09:14<00:00]        

[SHAP] Saved: C:\Users\Admin\Documents\WBS\Dissertation\Submission Related files\Notebooks\Modelling\outputs\20250827_144050_shap_global_kmeans_2.csv and C:\Users\Admin\Documents\WBS\Dissertation\Submission Related files\Notebooks\Modelling\outputs\20250827_144050_shap_top_features_per_cluster_kmeans_2.csv


100%|===================| 5994/6000 [16:27<00:00]        

[SHAP] Saved beeswarm: C:\Users\Admin\Documents\WBS\Dissertation\Submission Related files\Notebooks\Modelling\outputs\20250827_144050_shap_beeswarm_kmeans_2.png
Note: SHAP indicates feature association with cluster assignments, not causality. Correlated features may share credit.
time: 1h 56min 38s (started: 2025-08-27 19:01:47 +01:00)


## 9. Model Evaluation & Identifying Personas

In [11]:
# 9). Model Evaluation & Identifying Personas

# 1) Imports
import numpy as np
import pandas as pd
from pathlib import Path
from IPython.display import display

# 2) Config & paths (pulled from global CONFIG block where possible)
BASELINE_LEVEL = globals().get("BASELINE_LEVEL", "session")        # "session" | "visitor"
MONETARY_MODE  = globals().get("MONETARY_MODE", "transactions")    # "transactions" | "none"
OUT_DIR        = globals().get("OUT_DIR", Path("outputs")); OUT_DIR.mkdir(parents=True, exist_ok=True)
PERSONA_OUT    = globals().get("PERSONA_OUT", OUT_DIR / "personas.csv")
TS_DIV         = float(globals().get("TS_DIV", 1))                 # respect timestamp units
RANDOM_SEED    = globals().get("RANDOM_SEED", 42)
RNG            = globals().get("RNG", np.random.default_rng(RANDOM_SEED))

# 3) Preconditions
assert "events" in globals() and isinstance(events, pd.DataFrame), "Missing 'events' dataframe from earlier blocks."
need_cols = {"session_id", "visitorid", "timestamp", "event"}
missing   = need_cols - set(events.columns)
assert not missing, f"'events' is missing columns: {missing}"

# 4) Build unit table (R/F/M inputs)
if BASELINE_LEVEL == "session":
    unit_ids  = events["session_id"].drop_duplicates().astype("int64")
    unit_tbl  = pd.DataFrame({"unit_id": unit_ids.values})
    sess_last = (events.groupby("session_id")["timestamp"].max() // TS_DIV)
    sess_freq = events.groupby("session_id").size()
    tx_counts = (events.assign(event=events["event"].astype(str).str.lower())
                      .loc[lambda d: d["event"] == "transaction"]
                      .groupby("session_id").size())
    unit_tbl = (unit_tbl
        .merge(sess_last.rename("last_ts"), left_on="unit_id", right_index=True, how="left")
        .merge(sess_freq.rename("frequency"), left_on="unit_id", right_index=True, how="left")
        .merge(tx_counts.rename("tx_count"), left_on="unit_id", right_index=True, how="left"))
else:  # visitor-level
    unit_ids = events["visitorid"].drop_duplicates()
    unit_tbl = pd.DataFrame({"unit_id": unit_ids.values})
    vis_last = (events.groupby("visitorid")["timestamp"].max() // TS_DIV)
    vis_freq = events.groupby("visitorid").size()
    tx_counts = (events.assign(event=events["event"].astype(str).str.lower())
                      .loc[lambda d: d["event"] == "transaction"]
                      .groupby("visitorid").size())
    unit_tbl = (unit_tbl
        .merge(vis_last.rename("last_ts"), left_on="unit_id", right_index=True, how="left")
        .merge(vis_freq.rename("frequency"), left_on="unit_id", right_index=True, how="left")
        .merge(tx_counts.rename("tx_count"), left_on="unit_id", right_index=True, how="left"))

# 5) Recency (days) & monetary
max_ts = int((events["timestamp"].max() // TS_DIV))
unit_tbl["recency_days"] = ((max_ts - unit_tbl["last_ts"]) / (60 * 60 * 24)).astype("float64")
unit_tbl["tx_count"]     = unit_tbl["tx_count"].fillna(0).astype("float64")
unit_tbl["monetary"]     = (1.0 if MONETARY_MODE == "none" else unit_tbl["tx_count"])

# 6) Attach RFM cluster labels (prefer labels from 'out', else fall back to labels_dict)
label_series = None

if BASELINE_LEVEL == "session":
    # try from 'out' (RFM KMeans block)
    if "out" in globals() and isinstance(out, pd.DataFrame) and ("rfm_proxy_label" in out.columns):
        lab_src = out["rfm_proxy_label"]
        if lab_src.index.name != "session_id":
            if "session_id" in out.columns:
                lab_src = pd.Series(lab_src.values, index=out["session_id"].values)
            elif "SESSION_INDEX" in globals():
                lab_src = pd.Series(lab_src.values, index=pd.Index(SESSION_INDEX, name="session_id"))
        label_series = lab_src.rename("label").astype(int, copy=False)

    # fallback to labels_dict
    if (label_series is None) and ("labels_dict" in globals()):
        rfm_keys = [k for k in labels_dict if k.startswith("rfm_proxy_kmeans_session_")]
        if rfm_keys:
            k0 = sorted(rfm_keys)[0]
            if "sess" in globals():
                idx = sess.index
            elif "SESSION_INDEX" in globals():
                idx = pd.Index(SESSION_INDEX, name="session_id")
            else:
                idx = events["session_id"].drop_duplicates().astype("int64")
            label_series = pd.Series(labels_dict[k0], index=idx, name="label").astype(int, copy=False)

    assert label_series is not None, "RFM session labels not found. Run Section 5.2 first."
    unit_lbl = unit_tbl.merge(label_series.rename_axis("session_id"),
                              left_on="unit_id", right_index=True, how="left")
else:
    # visitor-level: derive majority label across that visitor's sessions
    if "out" in globals() and isinstance(out, pd.DataFrame) and ("rfm_proxy_label" in out.columns):
        sess_lab = out["rfm_proxy_label"]
        if sess_lab.index.name != "session_id":
            if "session_id" in out.columns:
                sess_lab = pd.Series(sess_lab.values, index=out["session_id"].values)
            elif "SESSION_INDEX" in globals():
                sess_lab = pd.Series(sess_lab.values, index=pd.Index(SESSION_INDEX, name="session_id"))
    else:
        assert "labels_dict" in globals(), "labels_dict not found; run the RFM KMeans block (5.2)."
        rfm_keys = [k for k in labels_dict if k.startswith("rfm_proxy_kmeans_session_")]
        assert rfm_keys, "No RFM session labels found in labels_dict."
        k0 = sorted(rfm_keys)[0]
        if "sess" in globals():
            idx = sess.index
        elif "SESSION_INDEX" in globals():
            idx = pd.Index(SESSION_INDEX, name="session_id")
        else:
            idx = events["session_id"].drop_duplicates().astype("int64")
        sess_lab = pd.Series(labels_dict[k0], index=idx, name="rfm_proxy_label")

    sess_to_vis = (events[["session_id", "visitorid"]]
                   .drop_duplicates()
                   .set_index("session_id")["visitorid"])
    tmp = pd.DataFrame({
        "visitorid": sess_to_vis.reindex(sess_lab.index).values,
        "label": sess_lab.values
    }).dropna()
    vis_lab = (tmp.groupby("visitorid")["label"]
                  .agg(lambda s: s.value_counts().idxmax())
                  .astype(int))
    unit_lbl = unit_tbl.merge(vis_lab.rename_axis("unit_id").rename("label"),
                              left_on="unit_id", right_index=True, how="left")

# 7) Event shares (views / add-to-cart / transactions)
evt = events[["session_id", "visitorid", "event"]].copy()
evt["event"] = evt["event"].astype(str).str.lower()
evt["is_view"]        = (evt["event"] == "view").astype(int)
evt["is_addtocart"]   = (evt["event"] == "addtocart").astype(int)
evt["is_transaction"] = (evt["event"] == "transaction").astype(int)

if BASELINE_LEVEL == "session":
    e_agg = (evt.groupby("session_id")[["is_view","is_addtocart","is_transaction"]]
                .sum().reset_index().rename(columns={"session_id":"unit_id"}))
else:
    e_agg = (evt.groupby("visitorid")[["is_view","is_addtocart","is_transaction"]]
                .sum().reset_index().rename(columns={"visitorid":"unit_id"}))
e_agg["events"] = e_agg[["is_view","is_addtocart","is_transaction"]].sum(axis=1)

prof = unit_lbl.merge(e_agg, on="unit_id", how="left")

# proportions (safe for zero denominators)
denom = prof["events"].replace(0, np.nan)
prof["p_view"]        = (prof["is_view"]        / denom).fillna(0.0)
prof["p_addtocart"]   = (prof["is_addtocart"]   / denom).fillna(0.0)
prof["p_transaction"] = (prof["is_transaction"] / denom).fillna(0.0)

# 8) Cluster-level summary
summary = (prof
    .groupby("label", dropna=False)
    .agg(
        n=("unit_id", "count"),
        share=("unit_id", lambda s: len(s) / max(1, len(prof))),   # added share for readability
        recency_days_median=("recency_days", "median"),
        frequency_median=("frequency", "median"),
        monetary_median=("monetary", "median"),
        events_mean=("events", "mean"),
        p_view_mean=("p_view", "mean"),
        p_addtocart_mean=("p_addtocart", "mean"),
        p_transaction_mean=("p_transaction", "mean"),
    )
    .reset_index()
    .sort_values(["n"], ascending=False)
)

# 9) Save profile & personas
summary_path = OUT_DIR / f"rfm_cluster_profile_{BASELINE_LEVEL}.csv"
summary.to_csv(summary_path, index=False)
summary.to_csv(PERSONA_OUT, index=False)

print("Baseline RFM profile (top rows):")
try:
    display(summary.head(10))
except Exception:
    print(summary.head(10).to_string(index=False))

print(f"Saved profile table : {summary_path}")
print(f"Saved personas to   : {PERSONA_OUT}")


Baseline RFM profile (top rows):


,label,n,share,recency_days_median,frequency_median,monetary_median,events_mean,p_view_mean,p_addtocart_mean,p_transaction_mean
1,1,145960,0.393322,0.105023,2.0,0.0,2.417505,1.000000,0.000000,0.000000
3,3,144785,0.390156,0.037303,2.0,0.0,2.440356,1.000000,0.000000,0.000000
0,0,41761,0.112535,0.078600,6.0,0.0,7.975647,0.999360,0.000640,0.000000
4,4,20639,0.055616,0.071343,3.0,0.0,3.490576,0.643163,0.356837,0.000000
5,5,14920,0.040205,0.069450,4.0,1.0,5.335925,0.495478,0.314757,0.189765
2,2,3030,0.008165,0.068519,11.0,2.0,14.258746,0.479642,0.332943,0.187415


Saved profile table : C:\Users\Admin\Documents\WBS\Dissertation\Submission Related files\Notebooks\Modelling\outputs\rfm_cluster_profile_session.csv
Saved personas to   : C:\Users\Admin\Documents\WBS\Dissertation\Submission Related files\Notebooks\Modelling\outputs\personas.csv
time: 3.83 s (started: 2025-08-27 20:58:25 +01:00)
